# Data Prep

## Version 1 - minimal subset of data (for sample visual)

In [ ]:
"""
Preprocessing and Cleaning for NYC EMS Data and Hospital Data
"""

import pandas as pd
import numpy as np
import geopandas as gpd
from datetime import datetime
import matplotlib.pyplot as plt
from shapely.geometry import Point
import warnings
import os
import traceback

# Suppress warnings
warnings.filterwarnings('ignore')

################################
# 1. DATA VALIDATION FUNCTIONS #
################################

def validate_ems_data(df):
    """Validate EMS data structure and make necessary corrections"""
    print("Validating EMS data structure...")

    # Check if columns need to be shifted
    if isinstance(df.columns[0], str) and '"' in df.columns[0]:
        print("Found unexpected format, attempting to fix column structure...")
        # Assume first column contains incident ID with a quote
        df['CAD_INCIDENT_ID'] = df.iloc[:, 0].str.replace('"', '')

        # Shift date columns to correct positions
        date_cols = [col for col in df.columns if 'T' in str(col) and ':' in str(col)]
        if len(date_cols) >= 2:
            df['INCIDENT_DATETIME'] = df[date_cols[0]]
            df['FIRST_ASSIGNMENT_DATETIME'] = df[date_cols[1]]

            if len(date_cols) > 2:
                df['FIRST_ON_SCENE_DATETIME'] = df[date_cols[2]]

        # Try to identify borough column
        for col in df.columns:
            if isinstance(col, str) and df[col].astype(str).str.contains('BRONX|MANHATTAN|BROOKLYN|QUEENS|STATEN', case=False).any():
                print(f"Found potential BOROUGH column: {col}")
                df['BOROUGH'] = df[col]
                break

        # Try to identify zipcode column
        for col in df.columns:
            # Check if column contains values that look like zipcodes
            if df[col].astype(str).str.match(r'^\d{5}$').any():
                print(f"Found potential ZIPCODE column: {col}")
                df['ZIPCODE'] = df[col]
                break

    return df

def parse_datetime_columns(df, timestamp_cols):
    """Try multiple datetime formats to parse columns"""
    date_formats = [
        '%Y-%m-%dT%H:%M:%S.%f',  # ISO format with milliseconds
        '%Y-%m-%dT%H:%M:%S',     # ISO format without milliseconds
        '%Y-%m-%d %H:%M:%S',     # Standard datetime
        '%m/%d/%Y %H:%M:%S'      # US format
    ]

    for col in timestamp_cols:
        if col in df.columns:
            for date_format in date_formats:
                try:
                    df[col] = pd.to_datetime(df[col], format=date_format, errors='coerce')
                    print(f"Successfully parsed {col} with format {date_format}")
                    break
                except:
                    continue

    return df

def add_nyc_locations(df):
    """Add synthetic NYC location data if missing"""
    # NYC borough coordinates (approximate centroids)
    borough_coords = {
        'MANHATTAN': (40.7831, -73.9712),
        'BRONX': (40.8448, -73.8648),
        'BROOKLYN': (40.6782, -73.9442),
        'QUEENS': (40.7282, -73.7949),
        'STATEN ISLAND': (40.5795, -74.1502)
    }

    # Add latitude and longitude columns
    if 'BOROUGH' in df.columns:
        # Use borough-based coordinates with small random variation
        df['LATITUDE'] = df['BOROUGH'].apply(
            lambda b: borough_coords.get(b, (40.7128, -74.0060))[0] + np.random.normal(0, 0.01)
        )
        df['LONGITUDE'] = df['BOROUGH'].apply(
            lambda b: borough_coords.get(b, (40.7128, -74.0060))[1] + np.random.normal(0, 0.01)
        )
    elif 'ZIPCODE' in df.columns:
        # Create a simple mapping from ZIP to coordinates (would be better with real data)
        # For now, just create random clusters around NYC
        zip_coords = {}
        for zip_code in df['ZIPCODE'].unique():
            # Random location in NYC area
            zip_coords[zip_code] = (
                40.7128 + np.random.normal(0, 0.05),
                -74.0060 + np.random.normal(0, 0.05)
            )

        df['LATITUDE'] = df['ZIPCODE'].apply(
            lambda z: zip_coords.get(z, (40.7128, -74.0060))[0] + np.random.normal(0, 0.003)
        )
        df['LONGITUDE'] = df['ZIPCODE'].apply(
            lambda z: zip_coords.get(z, (40.7128, -74.0060))[1] + np.random.normal(0, 0.003)
        )
    else:
        # Use NYC center with random distribution
        df['LATITUDE'] = 40.7128 + np.random.normal(0, 0.05, size=len(df))
        df['LONGITUDE'] = -74.0060 + np.random.normal(0, 0.05, size=len(df))

    return df

def add_default_emergency_types(df):
    """Add default emergency types for modeling if missing"""
    if 'INITIAL_CALL_TYPE' not in df.columns or df['INITIAL_CALL_TYPE'].nunique() <= 1:
        print("Creating default emergency types...")

        # Common NYC EMS call types
        default_call_types = [
            'DIFFICULTY BREATHING', 'CARDIAC CONDITION', 'ALTERED MENTAL STATUS',
            'TRAUMA', 'UNCONSCIOUS', 'CHEST PAIN', 'GENERAL ILLNESS'
        ]

        # Assign randomly based on borough or zipcode distribution
        if 'BOROUGH' in df.columns and df['BOROUGH'].nunique() > 1:
            # Different distribution by borough
            borough_call_map = {
                'MANHATTAN': ['CARDIAC CONDITION', 'DIFFICULTY BREATHING', 'TRAUMA'],
                'BRONX': ['DIFFICULTY BREATHING', 'TRAUMA', 'GENERAL ILLNESS'],
                'BROOKLYN': ['GENERAL ILLNESS', 'CARDIAC CONDITION', 'DIFFICULTY BREATHING'],
                'QUEENS': ['CHEST PAIN', 'TRAUMA', 'GENERAL ILLNESS'],
                'STATEN ISLAND': ['CARDIAC CONDITION', 'GENERAL ILLNESS', 'CHEST PAIN']
            }

            df['INITIAL_CALL_TYPE'] = df['BOROUGH'].apply(
                lambda b: np.random.choice(
                    borough_call_map.get(str(b).upper(), default_call_types),
                    p=[0.4, 0.3, 0.3] if str(b).upper() in borough_call_map else None
                )
            )
        else:
            # Random distribution
            df['INITIAL_CALL_TYPE'] = np.random.choice(
                default_call_types,
                size=len(df),
                p=[0.25, 0.2, 0.15, 0.15, 0.1, 0.1, 0.05]
            )

        # Add severity levels
        df['INITIAL_SEVERITY_LEVEL_CODE'] = np.random.choice(
            ['1', '2', '3', '4', '5', '6', '7', '8', '9'],
            size=len(df),
            p=[0.05, 0.10, 0.15, 0.20, 0.20, 0.15, 0.10, 0.03, 0.02]
        )

    return df

################################
# 2. NYC EMS DATA PREPROCESSING #
################################

def preprocess_nyc_ems_data_enhanced(df):
    """
    Preprocess the NYC EMS data with enhanced validation

    Parameters:
    df (pd.DataFrame): Raw EMS data

    Returns:
    pd.DataFrame: Cleaned and preprocessed EMS data
    pd.DataFrame: Aggregated data for Poisson model
    """
    print(f"Preprocessing NYC EMS data...")
    print(f"Working with {len(df)} EMS incidents")

    # Print the original column names for debugging
    print("\nCurrent column names:")
    print(df.columns.tolist())

    # Try to map columns
    required_cols = [
        'CAD_INCIDENT_ID', 'INCIDENT_DATETIME', 'INITIAL_CALL_TYPE',
        'INITIAL_SEVERITY_LEVEL_CODE', 'BOROUGH', 'ZIPCODE',
        'LATITUDE', 'LONGITUDE'
    ]

    # Check missing columns
    missing_cols = [col for col in required_cols if col not in df.columns]
    if missing_cols:
        print(f"Missing columns: {missing_cols}")

        # Try to create missing columns from existing data
        if 'CAD_INCIDENT_ID' in missing_cols and df.shape[1] > 0:
            df['CAD_INCIDENT_ID'] = range(1, len(df) + 1)

        if 'INCIDENT_DATETIME' in missing_cols:
            # Try to find a datetime-like column
            date_cols = [col for col in df.columns if
                         isinstance(col, str) and ('date' in col.lower() or 'time' in col.lower())]
            if date_cols:
                df['INCIDENT_DATETIME'] = df[date_cols[0]]
            else:
                # Create default using current date with random hours
                df['INCIDENT_DATETIME'] = pd.Timestamp('2023-01-01') + pd.to_timedelta(
                    np.random.randint(0, 24*30, size=len(df)), unit='h')

    # Define timestamp columns
    timestamp_cols = [
        'INCIDENT_DATETIME', 'FIRST_ASSIGNMENT_DATETIME',
        'FIRST_ACTIVATION_DATETIME', 'FIRST_ON_SCENE_DATETIME',
        'FIRST_TO_HOSP_DATETIME', 'FIRST_HOSP_ARRIVAL_DATETIME',
        'INCIDENT_CLOSE_DATETIME'
    ]

    # Process datetime columns with error handling
    print("\nProcessing datetime columns...")
    df = parse_datetime_columns(df, timestamp_cols)

    # Handle missing values
    print("\nHandling missing values...")
    na_counts = df.isna().sum()
    print(f"Columns with missing values: {na_counts[na_counts > 0].head()}")

    # Critical columns we need
    critical_cols = ['INCIDENT_DATETIME', 'INITIAL_CALL_TYPE', 'INITIAL_SEVERITY_LEVEL_CODE',
                     'ZIPCODE', 'BOROUGH']

    # Check if critical columns exist
    missing_critical_cols = [col for col in critical_cols if col not in df.columns]
    if missing_critical_cols:
        print(f"WARNING: Still missing critical columns after validation: {missing_critical_cols}")
        # Create placeholder columns
        for col in missing_critical_cols:
            print(f"Creating placeholder column for {col}")
            if col == 'INCIDENT_DATETIME':
                df[col] = pd.Timestamp('2023-01-01') + pd.to_timedelta(
                    np.random.randint(0, 24*30, size=len(df)), unit='h')
            elif col == 'INITIAL_CALL_TYPE':
                df[col] = np.random.choice(
                    ['DIFFICULTY BREATHING', 'CARDIAC CONDITION', 'TRAUMA', 'GENERAL ILLNESS'],
                    size=len(df))
            elif col == 'INITIAL_SEVERITY_LEVEL_CODE':
                df[col] = np.random.choice(['1', '2', '3', '4', '5'], size=len(df))
            elif col == 'ZIPCODE':
                df[col] = np.random.choice(['10001', '10002', '10003', '10004', '10005'], size=len(df))
            elif col == 'BOROUGH':
                df[col] = np.random.choice(
                    ['MANHATTAN', 'BRONX', 'BROOKLYN', 'QUEENS', 'STATEN ISLAND'],
                    size=len(df))

    # Drop rows with missing values in critical columns
    df_clean = df.dropna(subset=[col for col in critical_cols if col in df.columns])
    print(f"Removed {len(df) - len(df_clean)} rows with missing critical data")

    # Clean and standardize text fields
    print("\nStandardizing text fields...")
    text_cols = ['INITIAL_CALL_TYPE', 'FINAL_CALL_TYPE', 'INITIAL_SEVERITY_LEVEL_CODE',
                'FINAL_SEVERITY_LEVEL_CODE', 'BOROUGH', 'INCIDENT_DISPATCH_AREA']

    for col in text_cols:
        if col in df_clean.columns:
            # Convert to uppercase and strip whitespace
            if df_clean[col].dtype == 'object':
                df_clean[col] = df_clean[col].astype(str).str.upper().str.strip()

    # Handle ZIP codes
    print("\nCleaning ZIP codes...")
    if 'ZIPCODE' in df_clean.columns:
        # Convert to string
        df_clean['ZIPCODE'] = df_clean['ZIPCODE'].astype(str)
        # Extract first 5 digits of ZIP code
        df_clean['ZIPCODE'] = df_clean['ZIPCODE'].str.extract(r'(\d{5})').iloc[:, 0]
        # Count missing after extraction
        missing_zip = df_clean['ZIPCODE'].isna().sum()
        print(f"Missing ZIP codes after extraction: {missing_zip}")
        # Fill missing with default
        if missing_zip > 0:
            df_clean['ZIPCODE'] = df_clean['ZIPCODE'].fillna('10001')

    # Ensure latitude and longitude exist
    if 'LATITUDE' not in df_clean.columns or 'LONGITUDE' not in df_clean.columns:
        print("Adding location data...")
        df_clean = add_nyc_locations(df_clean)

    # Handle response time anomalies
    print("\nCleaning response time metrics...")
    time_cols = ['DISPATCH_RESPONSE_SECONDS_QY', 'INCIDENT_RESPONSE_SECONDS_QY',
                'INCIDENT_TRAVEL_TM_SECONDS_QY']

    for col in time_cols:
        if col in df_clean.columns:
            # Convert to numeric if not already
            if df_clean[col].dtype == 'object':
                df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')

            # Count negative values
            neg_count = (df_clean[col] < 0).sum()
            if neg_count > 0:
                print(f"Found {neg_count} negative values in {col}")
                # Set negative values to NaN
                df_clean.loc[df_clean[col] < 0, col] = np.nan

            # Count extreme values (>1 hour)
            extreme_count = (df_clean[col] > 3600).sum()
            if extreme_count > 0:
                print(f"Found {extreme_count} extreme values (>1 hour) in {col}")
                # Cap at 99.5 percentile
                q_high = df_clean[col].quantile(0.995)
                df_clean.loc[df_clean[col] > q_high, col] = q_high

    # Create emergency type feature
    print("\nCreating emergency type feature...")
    # 1. First try final call type and severity
    if 'FINAL_CALL_TYPE' in df_clean.columns and 'FINAL_SEVERITY_LEVEL_CODE' in df_clean.columns:
        # Fill missing final values with initial values
        df_clean['FINAL_CALL_TYPE'] = df_clean['FINAL_CALL_TYPE'].fillna(df_clean['INITIAL_CALL_TYPE'])
        df_clean['FINAL_SEVERITY_LEVEL_CODE'] = df_clean['FINAL_SEVERITY_LEVEL_CODE'].fillna(df_clean['INITIAL_SEVERITY_LEVEL_CODE'])
        # Create emergency type
        df_clean['EMERGENCY_TYPE'] = df_clean['FINAL_CALL_TYPE'] + '_' + df_clean['FINAL_SEVERITY_LEVEL_CODE'].astype(str)
    else:
        # Use initial values
        df_clean['EMERGENCY_TYPE'] = df_clean['INITIAL_CALL_TYPE'] + '_' + df_clean['INITIAL_SEVERITY_LEVEL_CODE'].astype(str)

    # Create time features
    print("\nCreating time features...")
    if 'INCIDENT_DATETIME' in df_clean.columns:
        # Extract time components
        df_clean['HOUR'] = df_clean['INCIDENT_DATETIME'].dt.hour
        df_clean['DAY'] = df_clean['INCIDENT_DATETIME'].dt.day
        df_clean['WEEKDAY'] = df_clean['INCIDENT_DATETIME'].dt.weekday
        df_clean['MONTH'] = df_clean['INCIDENT_DATETIME'].dt.month
        df_clean['YEAR'] = df_clean['INCIDENT_DATETIME'].dt.year

        # Create time periods
        time_periods = [(0, 6, 'NIGHT'), (6, 12, 'MORNING'),
                        (12, 18, 'AFTERNOON'), (18, 24, 'EVENING')]

        df_clean['TIME_PERIOD'] = 'UNKNOWN'
        for start, end, label in time_periods:
            mask = (df_clean['HOUR'] >= start) & (df_clean['HOUR'] < end)
            df_clean.loc[mask, 'TIME_PERIOD'] = label

        # Is weekend flag
        df_clean['IS_WEEKEND'] = df_clean['WEEKDAY'].isin([5, 6]).astype(int)

    # Aggregate for Poisson model
    print("\nAggregating data for Poisson model...")
    try:
        groupby_cols = ['ZIPCODE', 'EMERGENCY_TYPE', 'TIME_PERIOD', 'IS_WEEKEND']

        # Add month if we have enough data
        if 'YEAR' in df_clean.columns:
            years_of_data = df_clean['YEAR'].nunique()
            if years_of_data >= 2:  # If we have multiple years, include month in grouping
                groupby_cols.append('MONTH')

        # Calculate lambda (average arrival rate) for each combination
        poisson_data = df_clean.groupby(groupby_cols).size().reset_index(name='COUNT')

        # Calculate mean (lambda) and variance of counts
        lambda_stats = poisson_data.groupby(['ZIPCODE', 'EMERGENCY_TYPE'])['COUNT'].agg(['mean', 'var']).reset_index()
        lambda_stats.rename(columns={'mean': 'LAMBDA', 'var': 'VARIANCE'}, inplace=True)

        # Check if variance ≈ mean (Poisson property)
        lambda_stats['VARIANCE_TO_MEAN_RATIO'] = lambda_stats['VARIANCE'] / lambda_stats['LAMBDA'].replace(0, 0.001)
        lambda_stats['VARIANCE_TO_MEAN_RATIO'] = lambda_stats['VARIANCE_TO_MEAN_RATIO'].clip(0, 10)

        print(f"Created {len(poisson_data)} location-emergency-time combinations")
        print(f"Created {len(lambda_stats)} lambda values for Poisson model")
    except Exception as e:
        print(f"Error during Poisson aggregation: {e}")
        # Create a simple aggregation as fallback
        try:
            simple_groupby = ['ZIPCODE']
            if 'EMERGENCY_TYPE' in df_clean.columns:
                simple_groupby.append('EMERGENCY_TYPE')

            poisson_data = df_clean.groupby(simple_groupby).size().reset_index(name='COUNT')
            lambda_stats = poisson_data.groupby(['ZIPCODE'])['COUNT'].agg(['mean', 'var']).reset_index()
            lambda_stats.rename(columns={'mean': 'LAMBDA', 'var': 'VARIANCE'}, inplace=True)
            lambda_stats['VARIANCE_TO_MEAN_RATIO'] = lambda_stats['VARIANCE'] / lambda_stats['LAMBDA'].replace(0, 0.001)
            lambda_stats['VARIANCE_TO_MEAN_RATIO'] = lambda_stats['VARIANCE_TO_MEAN_RATIO'].clip(0, 10)
            print(f"Created {len(lambda_stats)} simple lambda values as fallback")
        except Exception as e2:
            print(f"Error during fallback aggregation: {e2}")
            # Create dummy lambda stats dataframe
            lambda_stats = pd.DataFrame({
                'ZIPCODE': df_clean['ZIPCODE'].unique()[:10],
                'EMERGENCY_TYPE': ['UNKNOWN'] * min(10, len(df_clean['ZIPCODE'].unique())),
                'LAMBDA': [1.0] * min(10, len(df_clean['ZIPCODE'].unique())),
                'VARIANCE': [1.0] * min(10, len(df_clean['ZIPCODE'].unique())),
                'VARIANCE_TO_MEAN_RATIO': [1.0] * min(10, len(df_clean['ZIPCODE'].unique()))
            })

    print("\nEMS data preprocessing complete!")
    return df_clean, lambda_stats

#############################
# 3. HOSPITAL PREPROCESSING #
#############################

def enhance_nyc_hospital_data(hospitals):
    """Add NYC-specific enhancements to hospital data"""

    # NYC trauma center mapping (based on actual NYC trauma centers)
    nyc_trauma_centers = {
        'BELLEVUE HOSPITAL CENTER': 'LEVEL 1',
        'KINGS COUNTY HOSPITAL CENTER': 'LEVEL 1',
        'JACOBI MEDICAL CENTER': 'LEVEL 1',
        'LINCOLN MEDICAL AND MENTAL HEALTH CENTER': 'LEVEL 1',
        'ELMHURST HOSPITAL CENTER': 'LEVEL 1',
        'RICHMOND UNIVERSITY MEDICAL CENTER': 'LEVEL 1',
        'NEW YORK-PRESBYTERIAN HOSPITAL': 'LEVEL 1',
        'HARLEM HOSPITAL CENTER': 'LEVEL 1',
        'STATEN ISLAND UNIVERSITY HOSPITAL': 'LEVEL 1',
        'MAIMONIDES MEDICAL CENTER': 'LEVEL 1',
        'NYU LANGONE MEDICAL CENTER': 'LEVEL 1',
        'JAMAICA HOSPITAL MEDICAL CENTER': 'LEVEL 1',
    }

    # Update trauma levels for known NYC trauma centers
    for hospital, trauma_level in nyc_trauma_centers.items():
        mask = hospitals['NAME'].str.contains(hospital, case=False, na=False)
        if mask.any():
            hospitals.loc[mask, 'TRAUMA_LEVEL'] = trauma_level
            hospitals.loc[mask, 'HAS_TRAUMA'] = 1

    return hospitals

def preprocess_hospital_data(hospital_filepath, nyc_only=True):
    """
    Preprocess the hospital data

    Parameters:
    hospital_filepath (str): Path to the hospital data file
    nyc_only (bool): Filter to only NYC hospitals

    Returns:
    gpd.GeoDataFrame: Cleaned and preprocessed hospital data
    """
    print(f"\nLoading and preprocessing hospital data from {hospital_filepath}...")

    try:
        # Load the data
        if hospital_filepath.endswith('.geojson'):
            hospitals = gpd.read_file(hospital_filepath)
        elif hospital_filepath.endswith('.csv'):
            hospitals = pd.read_csv(hospital_filepath)
            # Convert to GeoDataFrame
            if all(col in hospitals.columns for col in ['LONGITUDE', 'LATITUDE']):
                # Use LONGITUDE and LATITUDE columns
                hospitals['geometry'] = hospitals.apply(
                    lambda row: Point(row['LONGITUDE'], row['LATITUDE']), axis=1)
                hospitals = gpd.GeoDataFrame(hospitals, geometry='geometry', crs="EPSG:4326")
            elif all(col in hospitals.columns for col in ['x', 'y']):
                # Use x and y columns
                hospitals['geometry'] = hospitals.apply(
                    lambda row: Point(row['x'], row['y']), axis=1)
                hospitals = gpd.GeoDataFrame(hospitals, geometry='geometry', crs="EPSG:4326")
        else:
            print(f"Unsupported file format: {hospital_filepath}")
            return None

        print(f"Successfully loaded {len(hospitals)} hospitals")

        # Data exploration summary
        print("\nHospital Data Summary:")
        if 'STATE' in hospitals.columns:
            print(f"States represented: {hospitals['STATE'].nunique()}")
        if 'TYPE' in hospitals.columns:
            print(f"Hospital types: {hospitals['TYPE'].value_counts().head()}")

        # Filter to NYC only if requested
        if nyc_only:
            print("\nFiltering to NYC area hospitals...")
            # First filter by NY state
            if 'STATE' in hospitals.columns:
                hospitals = hospitals[hospitals['STATE'] == 'NY']
                print(f"Hospitals in NY state: {len(hospitals)}")

                # Then filter by NYC counties
                if 'COUNTY' in hospitals.columns:
                    nyc_counties = ['KINGS', 'QUEENS', 'NEW YORK', 'BRONX', 'RICHMOND']
                    hospitals = hospitals[hospitals['COUNTY'].str.upper().isin(nyc_counties)]
                    print(f"Hospitals in NYC counties: {len(hospitals)}")

            # If too few hospitals found, use bounding box as fallback
            if len(hospitals) < 50:
                print("Using bounding box to filter NYC hospitals...")
                # NYC bounding box
                nyc_bbox = {
                    'min_lat': 40.477399, 'max_lat': 40.917577,
                    'min_lon': -74.259090, 'max_lon': -73.700272
                }

                # Apply bounding box filter
                if all(col in hospitals.columns for col in ['LATITUDE', 'LONGITUDE']):
                    bbox_filter = (
                        (hospitals['LATITUDE'] >= nyc_bbox['min_lat']) &
                        (hospitals['LATITUDE'] <= nyc_bbox['max_lat']) &
                        (hospitals['LONGITUDE'] >= nyc_bbox['min_lon']) &
                        (hospitals['LONGITUDE'] <= nyc_bbox['max_lon'])
                    )
                    hospitals = hospitals[bbox_filter]
                    print(f"Hospitals in NYC bounding box: {len(hospitals)}")

        # Handle missing values in critical columns
        print("\nHandling missing values in critical columns...")
        # Check for missing values in critical columns
        critical_cols = ['ID', 'NAME', 'BEDS', 'TRAUMA', 'LATITUDE', 'LONGITUDE', 'ZIP']
        critical_cols_present = [col for col in critical_cols if col in hospitals.columns]
        na_counts = {col: hospitals[col].isna().sum() for col in critical_cols_present}
        print(f"Missing values in critical columns: {na_counts}")

        # Fill missing IDs
        if 'ID' not in hospitals.columns or hospitals['ID'].isna().any():
            print("Creating or fixing hospital IDs...")
            if 'ID' not in hospitals.columns:
                hospitals['ID'] = range(1, len(hospitals) + 1)
            else:
                hospitals['ID'] = hospitals['ID'].fillna(
                    hospitals['OBJECTID'].astype(str) if 'OBJECTID' in hospitals.columns
                    else pd.Series(range(len(hospitals))).astype(str)
                )

        # Create NAME if missing
        if 'NAME' not in hospitals.columns:
            print("Creating hospital names...")
            hospitals['NAME'] = [f"HOSPITAL_{i}" for i in range(1, len(hospitals) + 1)]

        # Fill missing bed counts
        if 'BEDS' not in hospitals.columns:
            print("Creating bed counts...")
            # Random bed counts based on hospital type
            if 'TYPE' in hospitals.columns:
                bed_means = {
                    'GENERAL ACUTE CARE': 250,
                    'CRITICAL ACCESS': 25,
                    'PSYCHIATRIC': 100,
                    'REHABILITATION': 80,
                    'LONG TERM CARE': 120
                }
                hospitals['BEDS'] = hospitals['TYPE'].apply(
                    lambda t: int(np.random.normal(
                        bed_means.get(t, 150),
                        bed_means.get(t, 150) * 0.3
                    ).clip(10, 1000))
                )
            else:
                hospitals['BEDS'] = np.random.randint(50, 500, size=len(hospitals))
        elif hospitals['BEDS'].isna().any():
            # Convert to numeric first
            hospitals['BEDS'] = pd.to_numeric(hospitals['BEDS'], errors='coerce')

            # Calculate median beds by type if type exists
            if 'TYPE' in hospitals.columns:
                median_beds_by_type = hospitals.groupby('TYPE')['BEDS'].median()

                # Fill missing values with median for that hospital type
                for hospital_type, median_beds in median_beds_by_type.items():
                    mask = (hospitals['TYPE'] == hospital_type) & (hospitals['BEDS'].isna())
                    hospitals.loc[mask, 'BEDS'] = median_beds

            # For any remaining NAs, use overall median
            hospitals['BEDS'] = hospitals['BEDS'].fillna(hospitals['BEDS'].median())

        # Ensure all bed values are at least 1
        hospitals['BEDS'] = pd.to_numeric(hospitals['BEDS'], errors='coerce')
        hospitals['BEDS'] = hospitals['BEDS'].fillna(100).astype(int).clip(lower=1)

        # Process trauma levels
        print("\nProcessing trauma levels...")
        if 'TRAUMA' not in hospitals.columns:
            print("Creating trauma level data...")
            # Assign trauma levels with probability
            hospitals['TRAUMA'] = np.random.choice(
                ['LEVEL 1', 'LEVEL 2', 'LEVEL 3', 'NOT AVAILABLE'],
                size=len(hospitals),
                p=[0.05, 0.10, 0.15, 0.70]
            )
        # Process trauma levels (continued)
        else:
            # Fill NAs with 'NOT AVAILABLE'
            hospitals['TRAUMA'] = hospitals['TRAUMA'].fillna('NOT AVAILABLE')

        # Standardize trauma levels
        trauma_mapping = {
            'LEVEL I': 'LEVEL 1',
            'LEVEL II': 'LEVEL 2',
            'LEVEL III': 'LEVEL 3',
            'LEVEL IV': 'LEVEL 4',
            'LEVEL 1': 'LEVEL 1',
            'LEVEL 2': 'LEVEL 2',
            'LEVEL 3': 'LEVEL 3',
            'LEVEL 4': 'LEVEL 4',
            'YES': 'UNKNOWN LEVEL',
            'TRUE': 'UNKNOWN LEVEL',
            'NO': 'NOT AVAILABLE',
            'FALSE': 'NOT AVAILABLE',
            '': 'NOT AVAILABLE',
            None: 'NOT AVAILABLE'
        }

        hospitals['TRAUMA_LEVEL'] = hospitals['TRAUMA'].map(
            lambda x: trauma_mapping.get(str(x).upper(), 'NOT AVAILABLE'))

        # Create has_trauma flag
        hospitals['HAS_TRAUMA'] = hospitals['TRAUMA_LEVEL'].apply(
            lambda x: 0 if x == 'NOT AVAILABLE' else 1)

        # Apply NYC-specific enhancements
        hospitals = enhance_nyc_hospital_data(hospitals)

        # Process helipad information
        print("\nProcessing helipad information...")
        if 'HELIPAD' not in hospitals.columns:
            print("Creating helipad data...")
            # Higher probability of helipad for trauma centers
            hospitals['HELIPAD'] = hospitals['HAS_TRAUMA'].apply(
                lambda x: np.random.choice([1, 0], p=[0.8, 0.2]) if x == 1
                else np.random.choice([1, 0], p=[0.1, 0.9])
            )
        else:
            # Clean existing helipad data
            helipad_mapping = {
                'Y': 1, 'YES': 1, 'TRUE': 1, '1': 1, 1: 1, True: 1,
                'N': 0, 'NO': 0, 'FALSE': 0, '0': 0, 0: 0, False: 0
            }
            hospitals['HELIPAD'] = hospitals['HELIPAD'].map(
                lambda x: helipad_mapping.get(x if isinstance(x, (int, bool)) else str(x).upper(), 0))

        # Estimate ER capacity
        print("\nCalculating ER capacity estimates...")
        if 'ER_CAPACITY' not in hospitals.columns:
            # Base on bed count and hospital type
            # Assumptions based on general hospital planning guidelines
            if 'TYPE' in hospitals.columns:
                er_ratio = {
                    'GENERAL ACUTE CARE': 0.15,
                    'CRITICAL ACCESS': 0.2,
                    'PSYCHIATRIC': 0.05,
                    'REHABILITATION': 0.01,
                    'LONG TERM CARE': 0.01,
                    'CHILDREN': 0.2
                }
                hospitals['ER_CAPACITY'] = hospitals.apply(
                    lambda row: max(5, int(row['BEDS'] * er_ratio.get(row['TYPE'], 0.1) *
                                          (1.3 if row['HAS_TRAUMA'] == 1 else 1.0))),
                    axis=1
                )
            else:
                # Simple formula based on beds
                hospitals['ER_CAPACITY'] = (hospitals['BEDS'] * 0.1).astype(int).clip(lower=5)

        # Calculate service radius for each hospital
        print("\nCalculating service radius for each hospital...")
        # Base on hospital size, trauma level, and ER capacity
        hospitals['SERVICE_RADIUS_KM'] = hospitals.apply(
            lambda row: 2.0 +  # Base radius
                        (row['BEDS'] / 1000) * 3.0 +  # Add for size
                        (row['HAS_TRAUMA'] * 2.0) +  # Add for trauma center
                        (row['HELIPAD'] * 1.0),  # Add for helipad
            axis=1
        )

        # Ensure all hospitals have required info for the model
        for col in ['ID', 'NAME', 'BEDS', 'TRAUMA_LEVEL', 'HAS_TRAUMA',
                   'HELIPAD', 'ER_CAPACITY', 'SERVICE_RADIUS_KM']:
            if col not in hospitals.columns:
                print(f"ERROR: Missing required column {col}")
                return None

        print("\nHospital data preprocessing complete!")
        return hospitals
    except Exception as e:
        print(f"Error during hospital data preprocessing: {e}")
        traceback.print_exc()
        return None

#####################################
# 4. TRAVEL TIME ESTIMATION MODULE #
#####################################

def calculate_travel_times(ems_data, hospital_data):
    """
    Calculate estimated travel times between incident locations and hospitals

    Parameters:
    ems_data (pd.DataFrame): Preprocessed EMS data
    hospital_data (gpd.GeoDataFrame): Preprocessed hospital data

    Returns:
    pd.DataFrame: Travel time estimates between ZIP codes and hospitals
    """
    print("\nCalculating travel times between incident locations and hospitals...")

    # Extract unique ZIP codes from EMS data
    zip_codes = ems_data['ZIPCODE'].unique()
    print(f"Found {len(zip_codes)} unique ZIP codes in EMS data")

    # Calculate centroid for each ZIP code
    print("Calculating centroid for each ZIP code...")
    zip_centroids = {}
    for zip_code in zip_codes:
        # Filter EMS data for this ZIP
        zip_data = ems_data[ems_data['ZIPCODE'] == zip_code]

        # Calculate centroid
        if 'LATITUDE' in zip_data.columns and 'LONGITUDE' in zip_data.columns:
            zip_centroids[zip_code] = (
                zip_data['LATITUDE'].mean(),
                zip_data['LONGITUDE'].mean()
            )
        else:
            # Default to NYC centroid with small randomization
            zip_centroids[zip_code] = (
                40.7128 + np.random.normal(0, 0.01),
                -74.0060 + np.random.normal(0, 0.01)
            )

    # Rest of the function remains the same...

    # Create travel time estimates
    print("Estimating travel times between ZIPs and hospitals...")

    # Calculate travel times using Haversine distance and average speed model
    def haversine_distance(lat1, lon1, lat2, lon2):
        """Calculate distance between points in km using Haversine formula"""
        R = 6371  # Earth radius in km

        # Convert degrees to radians
        lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])

        # Haversine formula
        dlat = lat2 - lat1
        dlon = lon2 - lon1
        a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
        c = 2 * np.arcsin(np.sqrt(a))

        return R * c

    # Model average speed based on time of day
    def estimate_travel_time(distance, time_period='TYPICAL'):
        """Estimate travel time in minutes based on distance and time period"""
        # Average speed in km/h by time period
        speeds = {
            'NIGHT': 40,  # Less traffic at night
            'MORNING': 25,  # Morning rush hour
            'AFTERNOON': 30,  # Afternoon traffic
            'EVENING': 25,  # Evening rush hour
            'TYPICAL': 30   # Default average
        }

        # Calculate time in minutes
        speed = speeds.get(time_period, speeds['TYPICAL'])
        time_mins = (distance / speed) * 60

        # Add baseline response time (traffic lights, etc.)
        baseline = 3 + (distance * 0.2)  # More intersections with longer distance

        return time_mins + baseline

    # Create travel time matrix
    travel_times = []

    for zip_code, (zip_lat, zip_lon) in zip_centroids.items():
        for _, hospital in hospital_data.iterrows():
            # Get hospital coordinates
            hosp_lat = hospital['LATITUDE']
            hosp_lon = hospital['LONGITUDE']

            # Calculate distance
            distance = haversine_distance(zip_lat, zip_lon, hosp_lat, hosp_lon)

            # Calculate travel times for different periods
            times = {}
            for period in ['NIGHT', 'MORNING', 'AFTERNOON', 'EVENING']:
                times[period] = estimate_travel_time(distance, period)

            # Add to results
            travel_times.append({
                'ZIPCODE': zip_code,
                'HOSPITAL_ID': hospital['ID'],
                'HOSPITAL_NAME': hospital['NAME'],
                'DISTANCE_KM': distance,
                'TRAVEL_TIME_NIGHT': times['NIGHT'],
                'TRAVEL_TIME_MORNING': times['MORNING'],
                'TRAVEL_TIME_AFTERNOON': times['AFTERNOON'],
                'TRAVEL_TIME_EVENING': times['EVENING'],
                'TRAVEL_TIME_AVG': np.mean(list(times.values()))
            })

    # Convert to DataFrame
    travel_df = pd.DataFrame(travel_times)
    print(f"Generated {len(travel_df)} travel time estimates")

    return travel_df

#################################
# 5. MAIN PREPROCESSING FUNCTION #
#################################

def preprocess_all_data(ems_filepath, hospital_filepath, output_dir='/content/drive/MyDrive/Semester 2/Optimization /Project/output'):
    """
    Preprocess both EMS and hospital data and save results

    Parameters:
    ems_filepath (str): Path to the EMS data file
    hospital_filepath (str): Path to the hospital data file
    output_dir (str): Directory to save output files
    """
    try:
        # Create output directory if it doesn't exist
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)

        # Parse original column names from file header
        if ems_filepath.endswith('.csv'):
            # Read the first row to get original column names
            with open(ems_filepath, 'r') as f:
                header = f.readline().strip()
                original_columns = header.split(',')
                print(f"Original column names: {original_columns}")

                # Convert to uppercase for consistency
                uppercase_columns = [col.upper() for col in original_columns]
                print(f"Column names after conversion to uppercase: {uppercase_columns}")

        # Load EMS data
        print(f"Loading and preprocessing NYC EMS data from {ems_filepath}...")
        try:
            ems_data = pd.read_csv(ems_filepath)
            print(f"Successfully loaded {len(ems_data)} EMS incidents")
        except Exception as e:
            print(f"Error loading EMS data: {e}")
            traceback.print_exc()
            print("Creating synthetic EMS data for testing...")

            # Define a comprehensive list of NYC ZIP codes
            nyc_zipcodes = [
                # Manhattan (10001-10282)
                '10001', '10002', '10003', '10004', '10005', '10006', '10007', '10009', '10010', '10011',
                '10012', '10013', '10014', '10016', '10017', '10018', '10019', '10020', '10021', '10022',
                '10023', '10024', '10025', '10026', '10027', '10028', '10029', '10030', '10031', '10032',
                '10033', '10034', '10035', '10036', '10037', '10038', '10039', '10040', '10044', '10065',
                '10069', '10075', '10128', '10280', '10282',

                # Bronx (10451-10475)
                '10451', '10452', '10453', '10454', '10455', '10456', '10457', '10458', '10459', '10460',
                '10461', '10462', '10463', '10464', '10465', '10466', '10467', '10468', '10469', '10470',
                '10471', '10472', '10473', '10474', '10475',

                # Brooklyn (11201-11256)
                '11201', '11203', '11204', '11205', '11206', '11207', '11208', '11209', '11210', '11211',
                '11212', '11213', '11214', '11215', '11216', '11217', '11218', '11219', '11220', '11221',
                '11222', '11223', '11224', '11225', '11226', '11228', '11229', '11230', '11231', '11232',
                '11233', '11234', '11235', '11236', '11237', '11238', '11239', '11249', '11256',

                # Queens (11101-11697)
                '11101', '11102', '11103', '11104', '11105', '11106', '11109', '11354', '11355', '11356',
                '11357', '11358', '11359', '11360', '11361', '11362', '11363', '11364', '11365', '11366',
                '11367', '11368', '11369', '11370', '11371', '11372', '11373', '11374', '11375', '11377',
                '11378', '11379', '11385', '11411', '11412', '11413', '11414', '11415', '11416', '11417',
                '11418', '11419', '11420', '11421', '11422', '11423', '11426', '11427', '11428', '11429',
                '11432', '11433', '11434', '11435', '11436', '11691', '11692', '11693', '11694', '11697',

                # Staten Island (10301-10314)
                '10301', '10302', '10303', '10304', '10305', '10306', '10307', '10308', '10309', '10310',
                '10311', '10312', '10314'
            ]

            # Create synthetic data with all NYC ZIP codes
            ems_data = pd.DataFrame({
                'CAD_INCIDENT_ID': range(1, 10001),  # Increase number of incidents
                'INCIDENT_DATETIME': pd.date_range(start='2023-01-01', periods=10000, freq='H'),
                'INITIAL_CALL_TYPE': np.random.choice(
                    ['CARDIAC', 'TRAUMA', 'RESPIRATORY', 'GENERAL', 'DIFFICULTY BREATHING',
                     'ALTERED MENTAL STATUS', 'UNCONSCIOUS', 'CHEST PAIN', 'GENERAL ILLNESS'],
                    size=10000),
                'INITIAL_SEVERITY_LEVEL_CODE': np.random.choice(['1', '2', '3', '4', '5'], size=10000),
                'BOROUGH': np.random.choice(
                    ['MANHATTAN', 'BRONX', 'BROOKLYN', 'QUEENS', 'STATEN ISLAND'], size=10000),
                'ZIPCODE': np.random.choice(nyc_zipcodes, size=10000)
            })

        # Identify format and preprocess
        print("\nEMS Data Summary:")
        try:
            if 'INCIDENT_DATETIME' in ems_data.columns:
                time_range = f"{ems_data['INCIDENT_DATETIME'].min()} to {ems_data['INCIDENT_DATETIME'].max()}"
                print(f"Time range: {time_range}")
            else:
                print("Error displaying time range: 'INCIDENT_DATETIME'")

            if 'INITIAL_CALL_TYPE' in ems_data.columns:
                top_call_types = ems_data['INITIAL_CALL_TYPE'].value_counts().head(5)
                print(f"Top call types: {top_call_types}")
            else:
                print("Error during data summary: 'INITIAL_CALL_TYPE'")
        except Exception as e:
            print(f"Error during data summary: {e}")

        # Validate and fix data structure if needed
        ems_data = validate_ems_data(ems_data)

        # Preprocess EMS data
        clean_ems_data, lambda_stats = preprocess_nyc_ems_data_enhanced(ems_data)

        # Preprocess hospital data
        hospital_data = preprocess_hospital_data(hospital_filepath)

        # Calculate travel times
        travel_times = calculate_travel_times(clean_ems_data, hospital_data)

        # Save processed data
        print("\nSaving processed data...")
        clean_ems_data.to_csv(f"{output_dir}/clean_ems_data.csv", index=False)
        lambda_stats.to_csv(f"{output_dir}/poisson_lambda_stats.csv", index=False)
        hospital_data.to_csv(f"{output_dir}/clean_hospital_data.csv", index=False)
        travel_times.to_csv(f"{output_dir}/travel_time_estimates.csv", index=False)

        print(f"\nPreprocessing complete! Files saved to {output_dir}")

    except Exception as e:
        print(f"Error during preprocessing: {e}")
        traceback.print_exc()

if __name__ == "__main__":
    # Example usage
    ems_filepath = "/content/drive/MyDrive/Semester 2/Optimization /Project/output/nyc_ems_data.csv"
    hospital_filepath = "/content/drive/MyDrive/Semester 2/Optimization /Project/output/hospitals.csv"

    preprocess_all_data(ems_filepath, hospital_filepath)

Original column names: ['"cad_incident_id"', '"incident_datetime"', '"initial_call_type"', '"initial_severity_level_code"', '"final_call_type"', '"final_severity_level_code"', '"first_assignment_datetime"', '"valid_dispatch_rspns_time_indc"', '"dispatch_response_seconds_qy"', '"first_activation_datetime"', '"first_on_scene_datetime"', '"valid_incident_rspns_time_indc"', '"incident_response_seconds_qy"', '"incident_travel_tm_seconds_qy"', '"first_to_hosp_datetime"', '"first_hosp_arrival_datetime"', '"incident_close_datetime"', '"held_indicator"', '"incident_disposition_code"', '"borough"', '"incident_dispatch_area"', '"zipcode"', '"policeprecinct"', '"citycouncildistrict"', '"communitydistrict"', '"communityschooldistrict"', '"congressionaldistrict"', '"reopen_indicator"', '"special_event_indicator"', '"standby_indicator"', '"transfer_indicator"']
Column names after conversion to uppercase: ['"CAD_INCIDENT_ID"', '"INCIDENT_DATETIME"', '"INITIAL_CALL_TYPE"', '"INITIAL_SEVERITY_LEVEL_CODE

## Version 2 - Full data prep

In [ ]:
"""
Preprocessing and Cleaning for NYC EMS Data and Hospital Data
"""

import pandas as pd
import numpy as np
import geopandas as gpd
from datetime import datetime
import matplotlib.pyplot as plt
from shapely.geometry import Point
import warnings
import os
import traceback

# Suppress warnings
warnings.filterwarnings('ignore')

################################
# 1. DATA VALIDATION FUNCTIONS #
################################

def validate_ems_data(df):
    """Validate EMS data structure and make necessary corrections"""
    print("Validating EMS data structure...")

    # Fix column names by removing quotes and converting to uppercase
    if any('"' in col for col in df.columns):
        print("Fixing quoted column names...")
        df.columns = [col.replace('"', '').upper() for col in df.columns]
    elif not any(col == col.upper() for col in df.columns):
        print("Converting column names to uppercase...")
        df.columns = [col.upper() for col in df.columns]

    print(f"Column names after cleaning: {df.columns.tolist()}")

    return df

def parse_datetime_columns(df, timestamp_cols):
    """Try multiple datetime formats to parse columns"""
    date_formats = [
        '%Y-%m-%dT%H:%M:%S.%f',  # ISO format with milliseconds
        '%Y-%m-%dT%H:%M:%S',     # ISO format without milliseconds
        '%Y-%m-%d %H:%M:%S',     # Standard datetime
        '%m/%d/%Y %H:%M:%S'      # US format
    ]

    for col in timestamp_cols:
        if col in df.columns:
            for date_format in date_formats:
                try:
                    df[col] = pd.to_datetime(df[col], format=date_format, errors='coerce')
                    print(f"Successfully parsed {col} with format {date_format}")
                    break
                except:
                    continue

    return df

def add_nyc_locations(df):
    """Add synthetic NYC location data if missing"""
    # NYC borough coordinates (approximate centroids)
    borough_coords = {
        'MANHATTAN': (40.7831, -73.9712),
        'BRONX': (40.8448, -73.8648),
        'BROOKLYN': (40.6782, -73.9442),
        'QUEENS': (40.7282, -73.7949),
        'STATEN ISLAND': (40.5795, -74.1502)
    }

    # Add latitude and longitude columns
    if 'BOROUGH' in df.columns:
        # Use borough-based coordinates with small random variation
        df['LATITUDE'] = df['BOROUGH'].apply(
            lambda b: borough_coords.get(str(b).upper(), (40.7128, -74.0060))[0] + np.random.normal(0, 0.01)
        )
        df['LONGITUDE'] = df['BOROUGH'].apply(
            lambda b: borough_coords.get(str(b).upper(), (40.7128, -74.0060))[1] + np.random.normal(0, 0.01)
        )
    elif 'ZIPCODE' in df.columns:
        # Create a simple mapping from ZIP to coordinates (would be better with real data)
        # For now, just create random clusters around NYC
        zip_coords = {}
        for zip_code in df['ZIPCODE'].unique():
            # Random location in NYC area
            zip_coords[zip_code] = (
                40.7128 + np.random.normal(0, 0.05),
                -74.0060 + np.random.normal(0, 0.05)
            )

        df['LATITUDE'] = df['ZIPCODE'].apply(
            lambda z: zip_coords.get(z, (40.7128, -74.0060))[0] + np.random.normal(0, 0.003)
        )
        df['LONGITUDE'] = df['ZIPCODE'].apply(
            lambda z: zip_coords.get(z, (40.7128, -74.0060))[1] + np.random.normal(0, 0.003)
        )
    else:
        # Use NYC center with random distribution
        df['LATITUDE'] = 40.7128 + np.random.normal(0, 0.05, size=len(df))
        df['LONGITUDE'] = -74.0060 + np.random.normal(0, 0.05, size=len(df))

    return df

def add_default_emergency_types(df):
    """Add default emergency types for modeling if missing"""
    if 'INITIAL_CALL_TYPE' not in df.columns or df['INITIAL_CALL_TYPE'].nunique() <= 1:
        print("Creating default emergency types...")

        # Common NYC EMS call types
        default_call_types = [
            'DIFFICULTY BREATHING', 'CARDIAC CONDITION', 'ALTERED MENTAL STATUS',
            'TRAUMA', 'UNCONSCIOUS', 'CHEST PAIN', 'GENERAL ILLNESS'
        ]

        # Assign randomly based on borough or zipcode distribution
        if 'BOROUGH' in df.columns and df['BOROUGH'].nunique() > 1:
            # Different distribution by borough
            borough_call_map = {
                'MANHATTAN': ['CARDIAC CONDITION', 'DIFFICULTY BREATHING', 'TRAUMA'],
                'BRONX': ['DIFFICULTY BREATHING', 'TRAUMA', 'GENERAL ILLNESS'],
                'BROOKLYN': ['GENERAL ILLNESS', 'CARDIAC CONDITION', 'DIFFICULTY BREATHING'],
                'QUEENS': ['CHEST PAIN', 'TRAUMA', 'GENERAL ILLNESS'],
                'STATEN ISLAND': ['CARDIAC CONDITION', 'GENERAL ILLNESS', 'CHEST PAIN']
            }

            df['INITIAL_CALL_TYPE'] = df['BOROUGH'].apply(
                lambda b: np.random.choice(
                    borough_call_map.get(str(b).upper(), default_call_types),
                    p=[0.4, 0.3, 0.3] if str(b).upper() in borough_call_map else None
                )
            )
        else:
            # Random distribution
            df['INITIAL_CALL_TYPE'] = np.random.choice(
                default_call_types,
                size=len(df),
                p=[0.25, 0.2, 0.15, 0.15, 0.1, 0.1, 0.05]
            )

        # Add severity levels
        df['INITIAL_SEVERITY_LEVEL_CODE'] = np.random.choice(
            ['1', '2', '3', '4', '5', '6', '7', '8', '9'],
            size=len(df),
            p=[0.05, 0.10, 0.15, 0.20, 0.20, 0.15, 0.10, 0.03, 0.02]
        )

    return df

################################
# 2. NYC EMS DATA PREPROCESSING #
################################

def preprocess_nyc_ems_data_enhanced(df):
    """
    Preprocess the NYC EMS data with enhanced validation

    Parameters:
    df (pd.DataFrame): Raw EMS data

    Returns:
    pd.DataFrame: Cleaned and preprocessed EMS data
    pd.DataFrame: Aggregated data for Poisson model
    """
    print(f"Preprocessing NYC EMS data...")
    print(f"Working with {len(df)} EMS incidents")

    # Print the original column names for debugging
    print("\nCurrent column names:")
    print(df.columns.tolist())

    # Try to map columns
    required_cols = [
        'CAD_INCIDENT_ID', 'INCIDENT_DATETIME', 'INITIAL_CALL_TYPE',
        'INITIAL_SEVERITY_LEVEL_CODE', 'BOROUGH', 'ZIPCODE',
        'LATITUDE', 'LONGITUDE'
    ]

    # Check missing columns
    missing_cols = [col for col in required_cols if col not in df.columns]
    if missing_cols:
        print(f"Missing columns: {missing_cols}")

        # Try to create missing columns from existing data
        if 'CAD_INCIDENT_ID' in missing_cols and 'cad_incident_id' in df.columns:
            df['CAD_INCIDENT_ID'] = df['cad_incident_id']
        elif 'CAD_INCIDENT_ID' in missing_cols and df.shape[1] > 0:
            df['CAD_INCIDENT_ID'] = range(1, len(df) + 1)

        # Map lowercase column names to uppercase if they exist
        for col in missing_cols:
            if col.lower() in df.columns:
                df[col] = df[col.lower()]
                print(f"Mapped {col.lower()} to {col}")

        if 'INCIDENT_DATETIME' in missing_cols and 'incident_datetime' in df.columns:
            df['INCIDENT_DATETIME'] = df['incident_datetime']
        elif 'INCIDENT_DATETIME' in missing_cols:
            # Try to find a datetime-like column
            date_cols = [col for col in df.columns if
                         isinstance(col, str) and ('date' in col.lower() or 'time' in col.lower())]
            if date_cols:
                df['INCIDENT_DATETIME'] = df[date_cols[0]]
            else:
                # Create default using current date with random hours
                df['INCIDENT_DATETIME'] = pd.Timestamp('2023-01-01') + pd.to_timedelta(
                    np.random.randint(0, 24*30, size=len(df)), unit='h')

    # Map critical lowercase columns to uppercase
    for col in ['initial_call_type', 'initial_severity_level_code', 'borough', 'zipcode']:
        if col in df.columns and col.upper() not in df.columns:
            df[col.upper()] = df[col]
            print(f"Mapped {col} to {col.upper()}")

    # Define timestamp columns
    timestamp_cols = [
        'INCIDENT_DATETIME', 'FIRST_ASSIGNMENT_DATETIME',
        'FIRST_ACTIVATION_DATETIME', 'FIRST_ON_SCENE_DATETIME',
        'FIRST_TO_HOSP_DATETIME', 'FIRST_HOSP_ARRIVAL_DATETIME',
        'INCIDENT_CLOSE_DATETIME'
    ]

    # Process datetime columns with error handling
    print("\nProcessing datetime columns...")
    df = parse_datetime_columns(df, timestamp_cols)

    # Handle missing values
    print("\nHandling missing values...")
    na_counts = df.isna().sum()
    print(f"Columns with missing values: {na_counts[na_counts > 0].head()}")

    # Critical columns we need
    critical_cols = ['INCIDENT_DATETIME', 'INITIAL_CALL_TYPE', 'INITIAL_SEVERITY_LEVEL_CODE',
                     'ZIPCODE', 'BOROUGH']

    # Check if critical columns exist
    missing_critical_cols = [col for col in critical_cols if col not in df.columns]
    if missing_critical_cols:
        print(f"WARNING: Still missing critical columns after validation: {missing_critical_cols}")
        # Create placeholder columns
        for col in missing_critical_cols:
            print(f"Creating placeholder column for {col}")
            if col == 'INCIDENT_DATETIME':
                df[col] = pd.Timestamp('2023-01-01') + pd.to_timedelta(
                    np.random.randint(0, 24*30, size=len(df)), unit='h')
            elif col == 'INITIAL_CALL_TYPE':
                df[col] = np.random.choice(
                    ['DIFFICULTY BREATHING', 'CARDIAC CONDITION', 'TRAUMA', 'GENERAL ILLNESS'],
                    size=len(df))
            elif col == 'INITIAL_SEVERITY_LEVEL_CODE':
                df[col] = np.random.choice(['1', '2', '3', '4', '5'], size=len(df))
            elif col == 'ZIPCODE':
                # Use all NYC ZIP codes for better distribution
                nyc_zipcodes = [
                    # Manhattan (10001-10282)
                    '10001', '10002', '10003', '10004', '10005', '10006', '10007', '10009', '10010', '10011',
                    '10012', '10013', '10014', '10016', '10017', '10018', '10019', '10020', '10021', '10022',
                    '10023', '10024', '10025', '10026', '10027', '10028', '10029', '10030', '10031', '10032',
                    '10033', '10034', '10035', '10036', '10037', '10038', '10039', '10040', '10044', '10065',
                    '10069', '10075', '10128', '10280', '10282',

                    # Bronx (10451-10475)
                    '10451', '10452', '10453', '10454', '10455', '10456', '10457', '10458', '10459', '10460',
                    '10461', '10462', '10463', '10464', '10465', '10466', '10467', '10468', '10469', '10470',
                    '10471', '10472', '10473', '10474', '10475',

                    # Brooklyn (11201-11256)
                    '11201', '11203', '11204', '11205', '11206', '11207', '11208', '11209', '11210', '11211',
                    '11212', '11213', '11214', '11215', '11216', '11217', '11218', '11219', '11220', '11221',
                    '11222', '11223', '11224', '11225', '11226', '11228', '11229', '11230', '11231', '11232',
                    '11233', '11234', '11235', '11236', '11237', '11238', '11239', '11249', '11256',

                    # Queens (11101-11697)
                    '11101', '11102', '11103', '11104', '11105', '11106', '11109', '11354', '11355', '11356',
                    '11357', '11358', '11359', '11360', '11361', '11362', '11363', '11364', '11365', '11366',
                    '11367', '11368', '11369', '11370', '11371', '11372', '11373', '11374', '11375', '11377',
                    '11378', '11379', '11385', '11411', '11412', '11413', '11414', '11415', '11416', '11417',
                    '11418', '11419', '11420', '11421', '11422', '11423', '11426', '11427', '11428', '11429',
                    '11432', '11433', '11434', '11435', '11436', '11691', '11692', '11693', '11694', '11697',

                    # Staten Island (10301-10314)
                    '10301', '10302', '10303', '10304', '10305', '10306', '10307', '10308', '10309', '10310',
                    '10311', '10312', '10314'
                ]
                df[col] = np.random.choice(nyc_zipcodes, size=len(df))
            elif col == 'BOROUGH':
                # Determine BOROUGH based on ZIPCODE if available
                if 'ZIPCODE' in df.columns:
                    # Create ZIP to borough mapping
                    zip_to_borough = {}
                    # Manhattan
                    for zip_code in ['10001', '10002', '10003', '10004', '10005', '10006', '10007', '10009', '10010',
                                     '10011', '10012', '10013', '10014', '10016', '10017', '10018', '10019', '10020',
                                     '10021', '10022', '10023', '10024', '10025', '10026', '10027', '10028', '10029',
                                     '10030', '10031', '10032', '10033', '10034', '10035', '10036', '10037', '10038',
                                     '10039', '10040', '10044', '10065', '10069', '10075', '10128', '10280', '10282']:
                        zip_to_borough[zip_code] = 'MANHATTAN'

                    # Bronx
                    for zip_code in ['10451', '10452', '10453', '10454', '10455', '10456', '10457', '10458', '10459',
                                     '10460', '10461', '10462', '10463', '10464', '10465', '10466', '10467', '10468',
                                     '10469', '10470', '10471', '10472', '10473', '10474', '10475']:
                        zip_to_borough[zip_code] = 'BRONX'

                    # Brooklyn
                    for zip_code in ['11201', '11203', '11204', '11205', '11206', '11207', '11208', '11209', '11210',
                                     '11211', '11212', '11213', '11214', '11215', '11216', '11217', '11218', '11219',
                                     '11220', '11221', '11222', '11223', '11224', '11225', '11226', '11228', '11229',
                                     '11230', '11231', '11232', '11233', '11234', '11235', '11236', '11237', '11238',
                                     '11239', '11249', '11256']:
                        zip_to_borough[zip_code] = 'BROOKLYN'

                    # Queens
                    for zip_code in ['11101', '11102', '11103', '11104', '11105', '11106', '11109', '11354', '11355',
                                     '11356', '11357', '11358', '11359', '11360', '11361', '11362', '11363', '11364',
                                     '11365', '11366', '11367', '11368', '11369', '11370', '11371', '11372', '11373',
                                     '11374', '11375', '11377', '11378', '11379', '11385', '11411', '11412', '11413',
                                     '11414', '11415', '11416', '11417', '11418', '11419', '11420', '11421', '11422',
                                     '11423', '11426', '11427', '11428', '11429', '11432', '11433', '11434', '11435',
                                     '11436', '11691', '11692', '11693', '11694', '11697']:
                        zip_to_borough[zip_code] = 'QUEENS'

                    # Staten Island
                    for zip_code in ['10301', '10302', '10303', '10304', '10305', '10306', '10307', '10308', '10309',
                                     '10310', '10311', '10312', '10314']:
                        zip_to_borough[zip_code] = 'STATEN ISLAND'

                    # Map ZIP codes to boroughs
                    df[col] = df['ZIPCODE'].map(zip_to_borough)

                    # Fill any missing values with random borough
                    mask = df[col].isna()
                    if mask.any():
                        df.loc[mask, col] = np.random.choice(
                            ['MANHATTAN', 'BRONX', 'BROOKLYN', 'QUEENS', 'STATEN ISLAND'],
                            size=mask.sum()
                        )
                else:
                    df[col] = np.random.choice(
                        ['MANHATTAN', 'BRONX', 'BROOKLYN', 'QUEENS', 'STATEN ISLAND'],
                        size=len(df))

    # Drop rows with missing values in critical columns
    df_clean = df.dropna(subset=[col for col in critical_cols if col in df.columns])
    print(f"Removed {len(df) - len(df_clean)} rows with missing critical data")

    # Clean and standardize text fields
    print("\nStandardizing text fields...")
    text_cols = ['INITIAL_CALL_TYPE', 'FINAL_CALL_TYPE', 'INITIAL_SEVERITY_LEVEL_CODE',
                'FINAL_SEVERITY_LEVEL_CODE', 'BOROUGH', 'INCIDENT_DISPATCH_AREA']

    for col in text_cols:
        if col in df_clean.columns:
            # Convert to uppercase and strip whitespace
            if df_clean[col].dtype == 'object':
                df_clean[col] = df_clean[col].astype(str).str.upper().str.strip()

    # Handle ZIP codes
    print("\nCleaning ZIP codes...")
    if 'ZIPCODE' in df_clean.columns:
        # Convert to string
        df_clean['ZIPCODE'] = df_clean['ZIPCODE'].astype(str)
        # Extract first 5 digits of ZIP code
        df_clean['ZIPCODE'] = df_clean['ZIPCODE'].str.extract(r'(\d{5})').iloc[:, 0]
        # Count missing after extraction
        missing_zip = df_clean['ZIPCODE'].isna().sum()
        print(f"Missing ZIP codes after extraction: {missing_zip}")
        # Fill missing with NYC ZIPs
        if missing_zip > 0:
            nyc_zipcodes = [
                '10001', '10002', '10003', '10004', '10005', '10006', '10007', '10009', '10010', '10011',
                '10451', '10452', '10453', '10454', '10455', '11201', '11203', '11204', '11205', '11206',
                '11101', '11102', '11103', '11104', '11105', '10301', '10302', '10303', '10304', '10305'
            ]
            df_clean.loc[df_clean['ZIPCODE'].isna(), 'ZIPCODE'] = np.random.choice(
                nyc_zipcodes, size=missing_zip)

    # Ensure latitude and longitude exist
    if 'LATITUDE' not in df_clean.columns or 'LONGITUDE' not in df_clean.columns:
        print("Adding location data...")
        df_clean = add_nyc_locations(df_clean)

    # Handle response time anomalies
    print("\nCleaning response time metrics...")
    time_cols = ['DISPATCH_RESPONSE_SECONDS_QY', 'INCIDENT_RESPONSE_SECONDS_QY',
                'INCIDENT_TRAVEL_TM_SECONDS_QY']

    for col in time_cols:
        if col in df_clean.columns:
            # Convert to numeric if not already
            if df_clean[col].dtype == 'object':
                df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')

            # Count negative values
            neg_count = (df_clean[col] < 0).sum()
            if neg_count > 0:
                print(f"Found {neg_count} negative values in {col}")
                # Set negative values to NaN
                df_clean.loc[df_clean[col] < 0, col] = np.nan

            # Count extreme values (>1 hour)
            extreme_count = (df_clean[col] > 3600).sum()
            if extreme_count > 0:
                print(f"Found {extreme_count} extreme values (>1 hour) in {col}")
                # Cap at 99.5 percentile
                q_high = df_clean[col].quantile(0.995)
                df_clean.loc[df_clean[col] > q_high, col] = q_high

    # Create emergency type feature
    print("\nCreating emergency type feature...")
    # 1. First try final call type and severity
    if 'FINAL_CALL_TYPE' in df_clean.columns and 'FINAL_SEVERITY_LEVEL_CODE' in df_clean.columns:
        # Fill missing final values with initial values
        df_clean['FINAL_CALL_TYPE'] = df_clean['FINAL_CALL_TYPE'].fillna(df_clean['INITIAL_CALL_TYPE'])
        df_clean['FINAL_SEVERITY_LEVEL_CODE'] = df_clean['FINAL_SEVERITY_LEVEL_CODE'].fillna(df_clean['INITIAL_SEVERITY_LEVEL_CODE'])
        # Create emergency type
        df_clean['EMERGENCY_TYPE'] = df_clean['FINAL_CALL_TYPE'] + '_' + df_clean['FINAL_SEVERITY_LEVEL_CODE'].astype(str)
    else:
        # Use initial values
        df_clean['EMERGENCY_TYPE'] = df_clean['INITIAL_CALL_TYPE'] + '_' + df_clean['INITIAL_SEVERITY_LEVEL_CODE'].astype(str)

    # Create time features
    print("\nCreating time features...")
    if 'INCIDENT_DATETIME' in df_clean.columns:
        # Extract time components
        df_clean['HOUR'] = df_clean['INCIDENT_DATETIME'].dt.hour
        df_clean['DAY'] = df_clean['INCIDENT_DATETIME'].dt.day
        df_clean['WEEKDAY'] = df_clean['INCIDENT_DATETIME'].dt.weekday
        df_clean['MONTH'] = df_clean['INCIDENT_DATETIME'].dt.month
        df_clean['YEAR'] = df_clean['INCIDENT_DATETIME'].dt.year

        # Create time periods
        time_periods = [(0, 6, 'NIGHT'), (6, 12, 'MORNING'),
                        (12, 18, 'AFTERNOON'), (18, 24, 'EVENING')]

        df_clean['TIME_PERIOD'] = 'UNKNOWN'
        for start, end, label in time_periods:
            mask = (df_clean['HOUR'] >= start) & (df_clean['HOUR'] < end)
            df_clean.loc[mask, 'TIME_PERIOD'] = label

        # Is weekend flag
        df_clean['IS_WEEKEND'] = df_clean['WEEKDAY'].isin([5, 6]).astype(int)

    # Aggregate for Poisson model
    print("\nAggregating data for Poisson model...")
    try:
        groupby_cols = ['ZIPCODE', 'EMERGENCY_TYPE', 'TIME_PERIOD', 'IS_WEEKEND']

        # Add month if we have enough data
        if 'YEAR' in df_clean.columns:
            years_of_data = df_clean['YEAR'].nunique()
            if years_of_data >= 2:  # If we have multiple years, include month in grouping
                groupby_cols.append('MONTH')

        # Calculate lambda (average arrival rate) for each combination
        poisson_data = df_clean.groupby(groupby_cols).size().reset_index(name='COUNT')

        # Calculate mean (lambda) and variance of counts
        lambda_stats = poisson_data.groupby(['ZIPCODE', 'EMERGENCY_TYPE'])['COUNT'].agg(['mean', 'var']).reset_index()
        lambda_stats.rename(columns={'mean': 'LAMBDA', 'var': 'VARIANCE'}, inplace=True)

        # Check if variance ≈ mean (Poisson property)
        lambda_stats['VARIANCE_TO_MEAN_RATIO'] = lambda_stats['VARIANCE'] / lambda_stats['LAMBDA'].replace(0, 0.001)
        lambda_stats['VARIANCE_TO_MEAN_RATIO'] = lambda_stats['VARIANCE_TO_MEAN_RATIO'].clip(0, 10)

        print(f"Created {len(poisson_data)} location-emergency-time combinations")
        print(f"Created {len(lambda_stats)} lambda values for Poisson model")
    except Exception as e:
        print(f"Error during Poisson aggregation: {e}")
        # Create a simple aggregation as fallback
        try:
            simple_groupby = ['ZIPCODE']
            if 'EMERGENCY_TYPE' in df_clean.columns:
                simple_groupby.append('EMERGENCY_TYPE')

            poisson_data = df_clean.groupby(simple_groupby).size().reset_index(name='COUNT')
            lambda_stats = poisson_data.groupby(['ZIPCODE'])['COUNT'].agg(['mean', 'var']).reset_index()
            lambda_stats.rename(columns={'mean': 'LAMBDA', 'var': 'VARIANCE'}, inplace=True)
            lambda_stats['VARIANCE_TO_MEAN_RATIO'] = lambda_stats['VARIANCE'] / lambda_stats['LAMBDA'].replace(0, 0.001)
            lambda_stats['VARIANCE_TO_MEAN_RATIO'] = lambda_stats['VARIANCE_TO_MEAN_RATIO'].clip(0, 10)
            print(f"Created {len(lambda_stats)} simple lambda values as fallback")
        except Exception as e2:
            print(f"Error during fallback aggregation: {e2}")
            # Create dummy lambda stats dataframe
            lambda_stats = pd.DataFrame({
                'ZIPCODE': df_clean['ZIPCODE'].unique()[:10],
                'EMERGENCY_TYPE': ['UNKNOWN'] * min(10, len(df_clean['ZIPCODE'].unique())),
                'LAMBDA': [1.0] * min(10, len(df_clean['ZIPCODE'].unique())),
                'VARIANCE': [1.0] * min(10, len(df_clean['ZIPCODE'].unique())),
                'VARIANCE_TO_MEAN_RATIO': [1.0] * min(10, len(df_clean['ZIPCODE'].unique()))
            })

    print("\nEMS data preprocessing complete!")
    return df_clean, lambda_stats

#############################
# 3. HOSPITAL PREPROCESSING #
#############################

def enhance_nyc_hospital_data(hospitals):
    """Add NYC-specific enhancements to hospital data"""

    # NYC trauma center mapping (based on actual NYC trauma centers)
    nyc_trauma_centers = {
        'BELLEVUE HOSPITAL CENTER': 'LEVEL 1',
        'KINGS COUNTY HOSPITAL CENTER': 'LEVEL 1',
        'JACOBI MEDICAL CENTER': 'LEVEL 1',
        'LINCOLN MEDICAL AND MENTAL HEALTH CENTER': 'LEVEL 1',
        'ELMHURST HOSPITAL CENTER': 'LEVEL 1',
        'RICHMOND UNIVERSITY MEDICAL CENTER': 'LEVEL 1',
        'NEW YORK-PRESBYTERIAN HOSPITAL': 'LEVEL 1',
        'HARLEM HOSPITAL CENTER': 'LEVEL 1',
        'STATEN ISLAND UNIVERSITY HOSPITAL': 'LEVEL 1',
        'MAIMONIDES MEDICAL CENTER': 'LEVEL 2',
        'NYU LANGONE MEDICAL CENTER': 'LEVEL 2',
        'MOUNT SINAI HOSPITAL': 'LEVEL 1',
        'MONTEFIORE MEDICAL CENTER': 'LEVEL 1'
    }

    # Fill in trauma center designations if we have them
    if 'TRAUMA' in hospitals.columns:
        # Convert trauma center names to uppercase for matching
        hospitals['NAME_UPPER'] = hospitals['NAME'].str.upper()

        # Map known trauma centers
        for hospital_name, trauma_level in nyc_trauma_centers.items():
            mask = hospitals['NAME_UPPER'].str.contains(hospital_name, na=False)
            if mask.any():
                hospitals.loc[mask, 'TRAUMA'] = trauma_level
                print(f"Mapped {hospital_name} as {trauma_level} trauma center")

        # Drop the uppercase name column
        hospitals.drop('NAME_UPPER', axis=1, inplace=True)
    else:
        hospitals['TRAUMA'] = 'UNKNOWN'

    # Add NYC capacity estimates based on actual data
    # NYC emergency department visit volume by borough (approximate annual visits)
    borough_ed_visits = {
        'MANHATTAN': 150000,  # Higher volume
        'BRONX': 130000,
        'BROOKLYN': 140000,
        'QUEENS': 120000,
        'STATEN ISLAND': 80000  # Lower volume
    }

    # Add ED capacity estimate if not present
    if 'ED_CAPACITY' not in hospitals.columns:
        # Default capacity based on hospital size
        hospitals['ED_CAPACITY'] = hospitals['BEDS'].apply(
            lambda beds: max(10, int(beds * 0.15))  # Estimate ED capacity as 15% of beds
        )

        # Adjust ED capacity based on borough if available
        if 'BOROUGH' in hospitals.columns:
            for borough, avg_visits in borough_ed_visits.items():
                borough_mask = hospitals['BOROUGH'] == borough
                if borough_mask.any():
                    # Scale capacity by borough volume
                    scale_factor = avg_visits / 120000  # Using 120000 as baseline
                    hospitals.loc[borough_mask, 'ED_CAPACITY'] = (
                        hospitals.loc[borough_mask, 'ED_CAPACITY'] * scale_factor
                    ).round().astype(int)

    # Add specialized capabilities
    if 'CAPABILITIES' not in hospitals.columns:
        hospitals['CAPABILITIES'] = ''

        # Assign capabilities based on hospital name and size
        hospitals.loc[hospitals['BEDS'] > 400, 'CAPABILITIES'] += 'CARDIAC,'
        hospitals.loc[hospitals['BEDS'] > 300, 'CAPABILITIES'] += 'STROKE,'
        hospitals.loc[hospitals['BEDS'] > 200, 'CAPABILITIES'] += 'PEDS,'

        # Specific hospitals known for burn treatment
        burn_centers = ['NEW YORK-PRESBYTERIAN', 'JACOBI', 'STATEN ISLAND UNIVERSITY']
        for center in burn_centers:
            hospitals.loc[hospitals['NAME'].str.contains(center, case=False, na=False), 'CAPABILITIES'] += 'BURN,'

        # Clean up capabilities string
        hospitals['CAPABILITIES'] = hospitals['CAPABILITIES'].str.rstrip(',')

    # Add helipad information if not present
    if 'HELIPAD' not in hospitals.columns:
        hospitals['HELIPAD'] = 0
        # Major trauma centers likely have helipads
        hospitals.loc[hospitals['TRAUMA'].isin(['LEVEL 1', 'LEVEL 2']), 'HELIPAD'] = 1
        # Large hospitals might have helipads
        hospitals.loc[(hospitals['BEDS'] > 500) & (hospitals['HELIPAD'] == 0), 'HELIPAD'] = 1

    return hospitals

def preprocess_hospital_data(df, nyc_only=True):
    """
    Preprocess hospital data for the EMS optimization model

    Parameters:
    df (pd.DataFrame): Raw hospital data
    nyc_only (bool): Filter to only NYC hospitals

    Returns:
    pd.DataFrame: Cleaned hospital data
    """
    print(f"Preprocessing hospital data...")
    print(f"Working with {len(df)} hospitals")

    # Print column names for debugging
    print("\nCurrent column names:")
    print(df.columns.tolist())

    # Process hospital names
    if 'NAME' in df.columns:
        print("\nProcessing hospital names...")
        # Convert to uppercase
        df['NAME'] = df['NAME'].str.upper()
        # Remove common suffixes
        common_suffixes = ['HOSPITAL', 'MEDICAL CENTER', 'HEALTH CENTER', 'HEALTHCARE']
        for suffix in common_suffixes:
            df['NAME'] = df['NAME'].str.replace(f" {suffix}$", "", regex=True)

    # Process location data
    print("\nProcessing location data...")
    # Required location columns
    location_cols = ['LATITUDE', 'LONGITUDE', 'STATE', 'COUNTY', 'ZIP']
    missing_loc_cols = [col for col in location_cols if col not in df.columns]

    if missing_loc_cols:
        print(f"Missing location columns: {missing_loc_cols}")
        # Try to map lowercase columns
        for col in missing_loc_cols:
            if col.lower() in df.columns:
                df[col] = df[col.lower()]

    # Filter to NYC area if requested
    if nyc_only:
        print("\nFiltering to NYC area hospitals...")
        # First filter to NY state
        if 'STATE' in df.columns:
            df = df[df['STATE'] == 'NY']
            print(f"Hospitals in NY state: {len(df)}")

        # Then filter to NYC counties
        if 'COUNTY' in df.columns:
            nyc_counties = ['NEW YORK', 'KINGS', 'QUEENS', 'BRONX', 'RICHMOND']
            df = df[df['COUNTY'].isin(nyc_counties)]
            print(f"Hospitals in NYC counties: {len(df)}")

        # Alternative: Filter by ZIP code if county not available
        elif 'ZIP' in df.columns:
            # NYC ZIP codes start with 100-104 (Manhattan), 112 (Brooklyn),
            # 104 (Bronx), 110-114 (Queens), 103 (Staten Island)
            df['ZIP'] = df['ZIP'].astype(str).str.extract(r'(\d{5})').iloc[:, 0]
            nyc_zip_prefixes = ['100', '101', '102', '103', '104', '110', '111', '112', '113', '114']
            df = df[df['ZIP'].str[:3].isin(nyc_zip_prefixes)]
            print(f"Hospitals filtered by NYC ZIP codes: {len(df)}")

    # Handle missing values in critical columns
    print("\nHandling missing values in critical columns...")
    critical_cols = ['ID', 'NAME', 'BEDS', 'TRAUMA', 'LATITUDE', 'LONGITUDE', 'ZIP']

    # Report missing values
    missing_vals = {col: df[col].isna().sum() if col in df.columns else 'column missing'
                    for col in critical_cols}
    print(f"Missing values in critical columns: {missing_vals}")

    # Generate IDs if missing
    if 'ID' not in df.columns or df['ID'].isna().any():
        print("Generating hospital IDs...")
        if 'ID' not in df.columns:
            df['ID'] = range(1, len(df) + 1)
        else:
            # Fill missing IDs
            missing_mask = df['ID'].isna()
            df.loc[missing_mask, 'ID'] = range(
                df['ID'].max() + 1,
                df['ID'].max() + 1 + missing_mask.sum()
            )

    # Process bed counts
    if 'BEDS' in df.columns:
        # Convert to numeric
        df['BEDS'] = pd.to_numeric(df['BEDS'], errors='coerce')
        # Fill missing with median based on hospital type
        if 'TYPE' in df.columns:
            for hosp_type in df['TYPE'].unique():
                median_beds = df.loc[df['TYPE'] == hosp_type, 'BEDS'].median()
                if not pd.isna(median_beds):
                    df.loc[(df['TYPE'] == hosp_type) & (df['BEDS'].isna()), 'BEDS'] = median_beds

        # Fill any remaining missing with overall median
        df['BEDS'] = df['BEDS'].fillna(df['BEDS'].median())
        # Ensure integer values
        df['BEDS'] = df['BEDS'].astype(int)
    else:
        # Create synthetic bed count based on hospital name
        print("Creating synthetic bed counts...")
        # Larger facilities tend to have words like "Medical Center" or "Hospital"
        df['BEDS'] = 100  # Default size
        if 'NAME' in df.columns:
            df.loc[df['NAME'].str.contains('MEDICAL CENTER', na=False), 'BEDS'] = 300
            df.loc[df['NAME'].str.contains('HOSPITAL', na=False), 'BEDS'] = 250
            df.loc[df['NAME'].str.contains('COMMUNITY', na=False), 'BEDS'] = 150
            df.loc[df['NAME'].str.contains('UNIVERSITY', na=False), 'BEDS'] = 400

    # Process trauma levels
    print("\nProcessing trauma levels...")
    if 'TRAUMA' not in df.columns or df['TRAUMA'].isna().all():
        df['TRAUMA'] = 'NONE'  # Default no trauma designation
    else:
        # Standardize trauma values
        df['TRAUMA'] = df['TRAUMA'].fillna('NONE')
        df['TRAUMA'] = df['TRAUMA'].astype(str).str.upper()
        # Map variations to standard formats
        trauma_map = {
            'Y': 'LEVEL 1',
            'YES': 'LEVEL 1',
            'TRUE': 'LEVEL 1',
            '1': 'LEVEL 1',
            'I': 'LEVEL 1',
            '2': 'LEVEL 2',
            'II': 'LEVEL 2',
            '3': 'LEVEL 3',
            'III': 'LEVEL 3',
            '4': 'LEVEL 4',
            'IV': 'LEVEL 4',
            'N': 'NONE',
            'NO': 'NONE',
            'FALSE': 'NONE',
            '0': 'NONE',
            'NA': 'NONE',
            'NAN': 'NONE',
            '': 'NONE'
        }

        for k, v in trauma_map.items():
            df.loc[df['TRAUMA'] == k, 'TRAUMA'] = v

    # Process helipad information
    print("\nProcessing helipad information...")
    if 'HELIPAD' not in df.columns:
        df['HELIPAD'] = 0  # Default no helipad
    else:
        # Convert to binary
        df['HELIPAD'] = df['HELIPAD'].fillna(0)
        # Map text values to binary
        helipad_map = {
            'Y': 1, 'YES': 1, 'TRUE': 1, '1': 1, 'T': 1,
            'N': 0, 'NO': 0, 'FALSE': 0, '0': 0, 'F': 0, '': 0
        }

        for k, v in helipad_map.items():
            if isinstance(k, str):
                df.loc[df['HELIPAD'].astype(str).str.upper() == k.upper(), 'HELIPAD'] = v
            else:
                df.loc[df['HELIPAD'] == k, 'HELIPAD'] = v

        # Convert to integer
        df['HELIPAD'] = df['HELIPAD'].astype(int)

    # Calculate ER capacity estimates
    print("\nCalculating ER capacity estimates...")
    if 'ER_CAPACITY' not in df.columns:
        # Estimate ER capacity based on hospital size
        df['ER_CAPACITY'] = np.round(df['BEDS'] * 0.2).astype(int)  # ~20% of beds for ER
        # Adjust for trauma centers
        df.loc[df['TRAUMA'] == 'LEVEL 1', 'ER_CAPACITY'] = np.round(df.loc[df['TRAUMA'] == 'LEVEL 1', 'ER_CAPACITY'] * 1.5).astype(int)
        df.loc[df['TRAUMA'] == 'LEVEL 2', 'ER_CAPACITY'] = np.round(df.loc[df['TRAUMA'] == 'LEVEL 2', 'ER_CAPACITY'] * 1.3).astype(int)
        # Minimum capacity
        df['ER_CAPACITY'] = df['ER_CAPACITY'].clip(lower=10)

    # Calculate service radius for each hospital
    print("\nCalculating service radius for each hospital...")
    if 'SERVICE_RADIUS' not in df.columns:
        # Calculate service radius based on hospital size and trauma level
        df['SERVICE_RADIUS'] = 2.0  # Default radius in miles

        # Adjust for hospital size
        df.loc[df['BEDS'] > 100, 'SERVICE_RADIUS'] = 3.0
        df.loc[df['BEDS'] > 300, 'SERVICE_RADIUS'] = 4.0
        df.loc[df['BEDS'] > 500, 'SERVICE_RADIUS'] = 5.0

        # Adjust for trauma designation
        df.loc[df['TRAUMA'] == 'LEVEL 1', 'SERVICE_RADIUS'] = df.loc[df['TRAUMA'] == 'LEVEL 1', 'SERVICE_RADIUS'] * 1.5
        df.loc[df['TRAUMA'] == 'LEVEL 2', 'SERVICE_RADIUS'] = df.loc[df['TRAUMA'] == 'LEVEL 2', 'SERVICE_RADIUS'] * 1.3

        # Higher density areas (Manhattan) have smaller radii
        if 'COUNTY' in df.columns:
            df.loc[df['COUNTY'] == 'NEW YORK', 'SERVICE_RADIUS'] = df.loc[df['COUNTY'] == 'NEW YORK', 'SERVICE_RADIUS'] * 0.8

    # Apply NYC-specific enhancements
    df = enhance_nyc_hospital_data(df)

    print("\nHospital data preprocessing complete!")
    return df

###############################
# 4. TRAVEL TIME CALCULATIONS #
###############################

def calculate_travel_times(ems_data, hospital_data):
    """
    Calculate travel times between incident locations and hospitals

    Parameters:
    ems_data (pd.DataFrame): Cleaned EMS data
    hospital_data (pd.DataFrame): Cleaned hospital data

    Returns:
    pd.DataFrame: Travel time matrix
    """
    print("Calculating travel times between incident locations and hospitals...")

    # Extract unique ZIP codes from EMS data
    if 'ZIPCODE' in ems_data.columns:
        zip_codes = ems_data['ZIPCODE'].unique()
        print(f"Found {len(zip_codes)} unique ZIP codes in EMS data")
    else:
        # Create representative ZIP codes
        print("No ZIP codes in EMS data, creating representative locations...")
        # Use borough centroids if available
        if 'BOROUGH' in ems_data.columns:
            boroughs = ems_data['BOROUGH'].unique()
            # NYC borough representative ZIP codes
            borough_zips = {
                'MANHATTAN': '10016',
                'BRONX': '10458',
                'BROOKLYN': '11217',
                'QUEENS': '11375',
                'STATEN ISLAND': '10314'
            }
            zip_codes = [borough_zips.get(b, '10016') for b in boroughs]
        else:
            # Default to major NYC ZIP codes
            zip_codes = ['10016', '10458', '11217', '11375', '10314']

    # Calculate centroid for each ZIP code
    print("Calculating centroid for each ZIP code...")
    zip_centroids = {}

    for zip_code in zip_codes:
        if 'ZIPCODE' in ems_data.columns:
            # Calculate actual centroid from data
            zip_data = ems_data[ems_data['ZIPCODE'] == zip_code]
            if 'LATITUDE' in zip_data.columns and 'LONGITUDE' in zip_data.columns and len(zip_data) > 0:
                lat = zip_data['LATITUDE'].mean()
                lon = zip_data['LONGITUDE'].mean()
                zip_centroids[zip_code] = (lat, lon)
            else:
                # Default Manhattan coordinates with slight variation
                base_lat, base_lon = 40.7128, -74.0060
                # Add variation based on ZIP code
                variation = int(zip_code[-2:]) / 100
                zip_centroids[zip_code] = (base_lat + variation, base_lon + variation/2)
        else:
            # Default Manhattan coordinates with slight variation
            base_lat, base_lon = 40.7128, -74.0060
            # Add variation based on ZIP code
            variation = int(zip_code[-2:]) / 100
            zip_centroids[zip_code] = (base_lat + variation, base_lon + variation/2)

    # Create travel time matrix
    print("Estimating travel times between ZIPs and hospitals...")
    travel_times = []

    # Typical speeds in miles per hour
    nyc_speed_estimates = {
        'MANHATTAN': 8.0,  # Slow due to congestion
        'BRONX': 12.0,
        'BROOKLYN': 10.0,
        'QUEENS': 15.0,
        'STATEN ISLAND': 20.0  # Less congestion
    }

    # Default speed in miles per hour
    default_speed = 12.0

    # Calculate travel time for each ZIP code to each hospital
    for zip_code, (zip_lat, zip_lon) in zip_centroids.items():
        for _, hospital in hospital_data.iterrows():
            if 'LATITUDE' in hospital and 'LONGITUDE' in hospital:
                hosp_lat = hospital['LATITUDE']
                hosp_lon = hospital['LONGITUDE']

                # Calculate Haversine distance
                # R = 3963.0  # Earth radius in miles
                # Calculate the great circle distance between two points
                # on the earth (specified in decimal degrees)

                # Convert decimal degrees to radians
                lat1, lon1, lat2, lon2 = map(np.radians, [zip_lat, zip_lon, hosp_lat, hosp_lon])

                # Haversine formula
                dlon = lon2 - lon1
                dlat = lat2 - lat1
                a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
                c = 2 * np.arcsin(np.sqrt(a))
                r = 3963.0  # Radius of earth in miles
                distance = c * r

                # Determine speed based on borough
                if 'COUNTY' in hospital:
                    county = hospital['COUNTY']
                    # Map county to borough
                    county_to_borough = {
                        'NEW YORK': 'MANHATTAN',
                        'KINGS': 'BROOKLYN',
                        'QUEENS': 'QUEENS',
                        'BRONX': 'BRONX',
                        'RICHMOND': 'STATEN ISLAND'
                    }
                    borough = county_to_borough.get(county, 'MANHATTAN')
                    speed = nyc_speed_estimates.get(borough, default_speed)
                else:
                    speed = default_speed

                # Calculate travel time in minutes
                travel_time = (distance / speed) * 60

                # Store result
                travel_times.append({
                    'ZIP': zip_code,
                    'HOSPITAL_ID': hospital['ID'],
                    'DISTANCE_MILES': distance,
                    'TRAVEL_TIME_MINUTES': travel_time
                })

    # Convert to DataFrame
    travel_time_df = pd.DataFrame(travel_times)
    print(f"Generated {len(travel_time_df)} travel time estimates")

    return travel_time_df

#######################
# 5. DATA INTEGRATION #
#######################

def integrate_and_save_data(ems_data, hospital_data, travel_time_data, lambda_stats, output_dir):
    """
    Integrate all preprocessed data and save to files

    Parameters:
    ems_data (pd.DataFrame): Cleaned EMS data
    hospital_data (pd.DataFrame): Cleaned hospital data
    travel_time_data (pd.DataFrame): Travel time matrix
    lambda_stats (pd.DataFrame): Poisson model parameters
    output_dir (str): Directory to save output files
    """
    print("Saving processed data...")

    # Create output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Save EMS data
    ems_data.to_csv(os.path.join(output_dir, 'new_ems_cleaned.csv'), index=False)

    # Save hospital data
    hospital_data.to_csv(os.path.join(output_dir, 'new_hospitals_cleaned.csv'), index=False)

    # Save travel time data
    travel_time_data.to_csv(os.path.join(output_dir, 'new_travel_times.csv'), index=False)

    # Save lambda statistics (Poisson model parameters)
    lambda_stats.to_csv(os.path.join(output_dir, 'new_lambda_stats.csv'), index=False)

    # Create a compact integrated dataset for optimization
    # Combine arrival rates with hospital capacities
    try:
        # Group lambda stats by ZIP code
        zip_lambdas = lambda_stats.groupby('ZIPCODE')['LAMBDA'].sum().reset_index()

        # Select key hospital attributes
        hospital_compact = hospital_data[['ID', 'NAME', 'BEDS', 'ER_CAPACITY', 'TRAUMA',
                                          'LATITUDE', 'LONGITUDE', 'SERVICE_RADIUS']].copy()

        # Create ZIP code to hospital mapping with travel times
        zip_hospital_mapping = travel_time_data[['ZIP', 'HOSPITAL_ID', 'TRAVEL_TIME_MINUTES']].copy()

        # Save integrated data
        zip_lambdas.to_csv(os.path.join(output_dir, 'zip_lambdas.csv'), index=False)
        hospital_compact.to_csv(os.path.join(output_dir, 'hospital_compact.csv'), index=False)
        zip_hospital_mapping.to_csv(os.path.join(output_dir, 'zip_hospital_mapping.csv'), index=False)
    except Exception as e:
        print(f"Error creating integrated dataset: {e}")
        traceback.print_exc()

    print(f"Preprocessing complete! Files saved to {output_dir}")

def main():
    """Main function to process all data"""
    try:
        # Set parameters
        ems_input_file = "/content/drive/MyDrive/Semester 2/Optimization /Project/output/nyc_ems_data.csv"  # Replace with actual path
        hospital_input_file = "/content/drive/MyDrive/Semester 2/Optimization /Project/output/hospitals.csv"  # Replace with actual path
        output_dir = "/content/drive/MyDrive/Semester 2/Optimization /Project/output"  # Replace with desired output directory

        # Load EMS data
        print(f"Loading and preprocessing NYC EMS data from {ems_input_file}...")
        try:
            ems_df = pd.read_csv(ems_input_file)
            print(f"Successfully loaded {len(ems_df)} EMS incidents")

            # Print a summary of the data
            print("\nEMS Data Summary:")
            try:
                print(f"Time range: {ems_df['INCIDENT_DATETIME'].min()} to {ems_df['INCIDENT_DATETIME'].max()}")
                print(f"Top call types: {ems_df['INITIAL_CALL_TYPE'].value_counts().head()}")
            except Exception as e:
                print(f"Error during data summary: {str(e)}")

            # Validate and clean EMS data
            ems_df = validate_ems_data(ems_df)

            # Preprocess EMS data
            cleaned_ems_df, lambda_stats = preprocess_nyc_ems_data_enhanced(ems_df)

        except Exception as e:
            print(f"Error processing EMS data: {e}")
            traceback.print_exc()
            # Create dummy data for testing
            print("Creating dummy EMS data...")
            cleaned_ems_df = pd.DataFrame({
                'ZIPCODE': np.random.choice(['10001', '10002', '10003', '10004', '10005'], 100),
                'LATITUDE': 40.7128 + np.random.normal(0, 0.01, 100),
                'LONGITUDE': -74.0060 + np.random.normal(0, 0.01, 100)
            })
            lambda_stats = pd.DataFrame({
                'ZIPCODE': ['10001', '10002', '10003', '10004', '10005'],
                'EMERGENCY_TYPE': ['GENERAL'] * 5,
                'LAMBDA': [10, 8, 12, 7, 9],
                'VARIANCE': [10, 8, 12, 7, 9],
                'VARIANCE_TO_MEAN_RATIO': [1.0, 1.0, 1.0, 1.0, 1.0]
            })

        # Load hospital data
        print(f"Loading and preprocessing hospital data from {hospital_input_file}...")
        try:
            hospitals_df = pd.read_csv(hospital_input_file)
            print(f"Successfully loaded {len(hospitals_df)} hospitals")

            # Print a summary of the data
            print("\nHospital Data Summary:")
            if 'STATE' in hospitals_df.columns:
                print(f"States represented: {hospitals_df['STATE'].nunique()}")
            if 'TYPE' in hospitals_df.columns:
                print(f"Hospital types: {hospitals_df['TYPE'].value_counts()}")

            # Preprocess hospital data
            cleaned_hospitals_df = preprocess_hospital_data(hospitals_df)

        except Exception as e:
            print(f"Error processing hospital data: {e}")
            traceback.print_exc()
            # Create dummy hospital data for testing
            print("Creating dummy hospital data...")
            cleaned_hospitals_df = pd.DataFrame({
                'ID': range(1, 11),
                'NAME': [f"HOSPITAL {i}" for i in range(1, 11)],
                'BEDS': np.random.randint(100, 500, 10),
                'ER_CAPACITY': np.random.randint(20, 100, 10),
                'TRAUMA': np.random.choice(['LEVEL 1', 'LEVEL 2', 'NONE'], 10),
                'LATITUDE': 40.7128 + np.random.normal(0, 0.05, 10),
                'LONGITUDE': -74.0060 + np.random.normal(0, 0.05, 10),
                'SERVICE_RADIUS': np.random.uniform(2.0, 5.0, 10)
            })

        # Calculate travel times
        travel_times_df = calculate_travel_times(cleaned_ems_df, cleaned_hospitals_df)

        # Save processed data
        integrate_and_save_data(cleaned_ems_df, cleaned_hospitals_df, travel_times_df, lambda_stats, output_dir)

    except Exception as e:
        print(f"Error in main processing: {e}")
        traceback.print_exc()

if __name__ == "__main__":
    main()

Loading and preprocessing NYC EMS data from /content/drive/MyDrive/Semester 2/Optimization /Project/output/nyc_ems_data.csv...
Error processing EMS data: [Errno 2] No such file or directory: '/content/drive/MyDrive/Semester 2/Optimization /Project/output/nyc_ems_data.csv'
Creating dummy EMS data...
Loading and preprocessing hospital data from /content/drive/MyDrive/Semester 2/Optimization /Project/output/hospitals.csv...
Error processing hospital data: [Errno 2] No such file or directory: '/content/drive/MyDrive/Semester 2/Optimization /Project/output/hospitals.csv'
Creating dummy hospital data...
Calculating travel times between incident locations and hospitals...
Found 5 unique ZIP codes in EMS data
Calculating centroid for each ZIP code...
Estimating travel times between ZIPs and hospitals...
Generated 50 travel time estimates
Saving processed data...
Preprocessing complete! Files saved to /content/drive/MyDrive/Semester 2/Optimization /Project/output


Traceback (most recent call last):
  File "<ipython-input-2-6384cc5b597c>", line 997, in main
    ems_df = pd.read_csv(ems_input_file)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pandas/io/parsers/readers.py", line 1026, in read_csv
    return _read(filepath_or_buffer, kwds)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pandas/io/parsers/readers.py", line 620, in _read
    parser = TextFileReader(filepath_or_buffer, **kwds)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pandas/io/parsers/readers.py", line 1620, in __init__
    self._engine = self._make_engine(f, self.engine)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pandas/io/parsers/readers.py", line 1880, in _make_engine
    self.handles = get_handle(
                   ^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pa

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# run model

## Version 0 - ignore

In [ ]:
import pandas as pd
import numpy as np
import cvxpy as cp
from scipy.stats import norm

# ----------------------------
# 1. LOAD DATA
# ----------------------------
output_dir = "/content/drive/MyDrive/Semester 2/Optimization /Project/output"
hospitals = pd.read_csv(f"{output_dir}/clean_hospital_data.csv")
lambda_stats = pd.read_csv(f"{output_dir}/poisson_lambda_stats.csv")
travel_times = pd.read_csv(f"{output_dir}/travel_time_estimates.csv")

# ----------------------------
# 2. BUILD INDEX SETS
# ----------------------------
ERs = list(hospitals["ID"])
incident_types = sorted(lambda_stats["EMERGENCY_TYPE"].unique())
locations = sorted(lambda_stats["ZIPCODE"].unique())

num_ers = len(ERs)
num_types = len(incident_types)
num_locs = len(locations)

# ----------------------------
# 3. PARAMETERS
# ----------------------------

# Capacities
C = hospitals.set_index("ID")["ER_CAPACITY"].reindex(ERs).values

# Poisson rates: λ[j, k] is rate for type j at location k
lambda_jk = np.zeros((num_types, num_locs))
for idx, row in lambda_stats.iterrows():
    j = incident_types.index(row["EMERGENCY_TYPE"])
    k = locations.index(row["ZIPCODE"])
    lambda_jk[j, k] = row["LAMBDA"]

# Travel times: t[i, k] is time from ER i to location k
t_ik = np.full((num_ers, num_locs), np.inf)
for idx, row in travel_times.iterrows():
    i = ERs.index(row["HOSPITAL_ID"])
    k = locations.index(row["ZIPCODE"])
    t_ik[i, k] = row["TRAVEL_TIME_AVG"]

# Specialty compatibility: assume all ERs can handle all types (set to 1)
m = np.ones((num_ers, num_types), dtype=int)

# Travel time threshold (minutes)
tau = 45

# ----------------------------
# 4. DECISION VARIABLES
# ----------------------------

x = cp.Variable((num_ers, num_types, num_locs), nonneg=True)

# ----------------------------
# 5. CONSTRAINTS
# ----------------------------

constraints = []

# Assignment constraints: all incidents of each type/location must be routed
for j in range(num_types):
    for k in range(num_locs):
        constraints.append(cp.sum(x[:, j, k]) == 1)

# Travel time and specialty constraints
for i in range(num_ers):
    for j in range(num_types):
        for k in range(num_locs):
            if t_ik[i, k] > tau or m[i, j] == 0:
                constraints.append(x[i, j, k] == 0)

# # ----------------------------
# # 6. OBJECTIVE: Maximize total slack with distance penalty
# # ----------------------------

# # Adjust weights to balance capacity and proximity
# alpha = 0.6  # Slack weight (capacity utilization)
# beta = 0.4   # Travel time weight

# # New slack calculation with logarithmic term to prevent over-assignment
# slack_terms = [cp.log(C[i] + 1 - cp.sum(cp.multiply(x[i,:,:], lambda_jk)))
#                for i in range(num_ers)]

# # Travel cost term weighted by incident volume
# travel_cost = cp.sum(cp.multiply(x, t_ik[:, None, :] * lambda_jk[None, :, :]))

# objective = cp.Maximize(alpha * cp.sum(slack_terms) - beta * travel_cost)
# # Weight parameter for distance penalty (tune this value)


# # ----------------------------
# # 7. SOLVE & PRINT RESULTS (updated)
# # ----------------------------

# prob = cp.Problem(objective, constraints)
# prob.solve(solver=cp.SCS, eps=1e-6)

# print("Optimal routing fractions (x):")
# total_travel_cost = 0
# for i, er in enumerate(ERs):
#     for j, typ in enumerate(incident_types):
#         for k, loc in enumerate(locations):
#             if x.value[i, j, k] > 1e-3:
#                 print(f"Send {x.value[i, j, k]:.2f} of {typ} at {loc} to {er}")
#                 # Calculate actual travel cost contribution
#                 total_travel_cost += x.value[i, j, k] * t_ik[i, k] * lambda_jk[j, k]

# print(f"\nTotal slack: {prob.value + beta * total_travel_cost:.2f}")
# print(f"Total travel cost: {total_travel_cost:.2f} minutes")
# print(f"Objective value (slack - β·travel): {prob.value:.2f}")


# ----------------------------
# 6. OBJECTIVE: Maximize total slack (capacity - expected load)
# ----------------------------

slacks = []
for i in range(num_ers):
    mu_i = cp.sum(cp.multiply(x[i, :, :], lambda_jk))
    slacks.append(C[i] - mu_i)
objective = cp.Maximize(cp.sum(slacks))

# ----------------------------
# 7. SOLVE & PRINT RESULTS
# ----------------------------

prob = cp.Problem(objective, constraints)
prob.solve(solver=cp.SCS, eps=1e-6)

print("Optimal routing fractions (x):")
for i, er in enumerate(ERs):
    for j, typ in enumerate(incident_types):
        for k, loc in enumerate(locations):
            if x.value[i, j, k] > 1e-3:
                print(f"Send {x.value[i, j, k]:.2f} of {typ} at {loc} to {er}")

print(f"\nObjective value (total slack): {prob.value:.2f}")

# ----------------------------
# 8. POST-SOLVE: Compute Probabilities
# ----------------------------

print("\n--- Probability each ER stays under capacity (Normal approx) ---")
for i, er in enumerate(ERs):
    mu_i = np.sum(x.value[i, :, :] * lambda_jk)
    sigma_i = np.sqrt(mu_i + 1e-6)
    z_score = (C[i] - mu_i) / (sigma_i + 1e-6)
    prob = norm.cdf(z_score)
    print(f"{er}: Probability under capacity ≈ {prob:.2%} (Expected load: {mu_i:.2f}, Capacity: {C[i]})")


Optimal routing fractions (x):
Send 0.01 of CARDIAC CONDITION_1 at 10001 to 15010456
Send 0.01 of CARDIAC CONDITION_1 at 10002 to 15010456
Send 0.01 of CARDIAC CONDITION_1 at 10003 to 15010456
Send 0.01 of CARDIAC CONDITION_1 at 10004 to 15010456
Send 0.01 of CARDIAC CONDITION_1 at 10005 to 15010456
Send 0.01 of CARDIAC CONDITION_2 at 10001 to 15010456
Send 0.01 of CARDIAC CONDITION_2 at 10002 to 15010456
Send 0.01 of CARDIAC CONDITION_2 at 10003 to 15010456
Send 0.01 of CARDIAC CONDITION_2 at 10004 to 15010456
Send 0.01 of CARDIAC CONDITION_2 at 10005 to 15010456
Send 0.01 of CARDIAC CONDITION_3 at 10001 to 15010456
Send 0.01 of CARDIAC CONDITION_3 at 10002 to 15010456
Send 0.01 of CARDIAC CONDITION_3 at 10003 to 15010456
Send 0.01 of CARDIAC CONDITION_3 at 10004 to 15010456
Send 0.01 of CARDIAC CONDITION_3 at 10005 to 15010456
Send 0.01 of CARDIAC CONDITION_4 at 10001 to 15010456
Send 0.01 of CARDIAC CONDITION_4 at 10002 to 15010456
Send 0.01 of CARDIAC CONDITION_4 at 10003 to 150104

In [ ]:
import pandas as pd
import numpy as np
import cvxpy as cp

# (Assume you have already defined ERs, incident_types, locations, lambda_jk, t_ik, C, m, tau, etc.)
# And have set up your decision variable x and constraints as in your previous code.

# # 1. Solve the problem
# prob = cp.Problem(objective, constraints)
# prob.solve(solver=cp.SCS, verbose=True)  # Use verbose=True to see solver output

# 2. Check status and extract numerical values
#if prob.status not in ["infeasible", "unbounded"]:
    # Get the routing fractions as a NumPy array
x_optimal = x.value
print("Routing fractions array:\n", x_optimal)

# Convert to a list for JSON/CSV export
x_list = x_optimal.tolist()

# 3. Export results to CSV
data = []
for i in range(len(ERs)):
    for j in range(len(incident_types)):
        for k in range(len(locations)):
            if x_optimal[i, j, k] > 1e-3:
                data.append({
                    'ER': ERs[i],
                    'Incident Type': incident_types[j],
                    'Location': locations[k],
                    'Fraction': x_optimal[i, j, k]
                })
df = pd.DataFrame(data)
df.to_csv('optimal_routing.csv', index=False)
print("Results exported to optimal_routing.csv")

# 4. Print objective value
print(f"\nObjective value: {prob.value:.2f}")

# else:
#     print(f"Problem status: {prob.status}. No feasible solution found.")

# 5. (Optional) Handle edge cases
# If you want to save the full array for large-scale problems:
# np.save('x_optimal.npy', x_optimal)


Routing fractions array:
 [[[0.01369863 0.01333333 0.01333333 0.01333333 0.01333333]
  [0.01369863 0.01333333 0.01333333 0.01333333 0.01333333]
  [0.01369863 0.01333333 0.01333333 0.01333333 0.01333333]
  ...
  [0.01369863 0.01333333 0.01333333 0.01333333 0.01333333]
  [0.01369863 0.01333333 0.01333333 0.01333333 0.01333333]
  [0.01369863 0.01333333 0.01333333 0.01333333 0.01333333]]

 [[0.01369863 0.01333333 0.01333333 0.01333333 0.01333333]
  [0.01369863 0.01333333 0.01333333 0.01333333 0.01333333]
  [0.01369863 0.01333333 0.01333333 0.01333333 0.01333333]
  ...
  [0.01369863 0.01333333 0.01333333 0.01333333 0.01333333]
  [0.01369863 0.01333333 0.01333333 0.01333333 0.01333333]
  [0.01369863 0.01333333 0.01333333 0.01333333 0.01333333]]

 [[0.         0.         0.         0.         0.        ]
  [0.         0.         0.         0.         0.        ]
  [0.         0.         0.         0.         0.        ]
  ...
  [0.         0.         0.         0.         0.        ]
  [0.   

AttributeError: 'numpy.float64' object has no attribute 'value'

## Version 1 - Minimal Subset of model

In [ ]:
import pandas as pd
import numpy as np
import cvxpy as cp
from scipy.stats import norm

# ----------------------------
# 1. LOAD DATA
# ----------------------------
output_dir = "/content/drive/MyDrive/Semester 2/Optimization /Project/output"
hospitals = pd.read_csv(f"{output_dir}/clean_hospital_data.csv")
lambda_stats = pd.read_csv(f"{output_dir}/poisson_lambda_stats.csv")
travel_times = pd.read_csv(f"{output_dir}/travel_time_estimates.csv")


# ----------------------------
# 2. BUILD INDEX SETS
# ----------------------------
ERs = list(hospitals["ID"])
incident_types = sorted(lambda_stats["EMERGENCY_TYPE"].unique())
locations = sorted(lambda_stats["ZIPCODE"].unique())
print(locations)
num_ers = len(ERs)
num_types = len(incident_types)
num_locs = len(locations)

# ----------------------------
# 3. PARAMETERS
# ----------------------------

# Capacities
C = hospitals.set_index("ID")["ER_CAPACITY"].reindex(ERs).values

# Poisson rates: λ[j, k] is rate for type j at location k
lambda_jk = np.zeros((num_types, num_locs))
for idx, row in lambda_stats.iterrows():
    j = incident_types.index(row["EMERGENCY_TYPE"])
    k = locations.index(row["ZIPCODE"])
    lambda_jk[j, k] = row["LAMBDA"]

# Travel times: t[i, k] is time from ER i to location k
t_ik = np.full((num_ers, num_locs), np.inf)
for idx, row in travel_times.iterrows():
    i = ERs.index(row["HOSPITAL_ID"])
    k = locations.index(row["ZIPCODE"])
    t_ik[i, k] = row["TRAVEL_TIME_AVG"]

# Specialty compatibility: assume all ERs can handle all types (set to 1)
m = np.ones((num_ers, num_types), dtype=int)

# Penalty weight for travel time (tune this parameter)
alpha = 0.001  # Adjusted for balance
beta = 0.1

# ----------------------------
# 4. DECISION VARIABLES
# ----------------------------

x = cp.Variable((num_ers, num_types, num_locs), nonneg=True)

# ----------------------------
# 5. CONSTRAINTS
# ----------------------------

constraints = []

# Assignment constraints: all incidents of each type/location must be routed
for j in range(num_types):
    for k in range(num_locs):
        constraints.append(cp.sum(x[:, j, k]) == 1)

# Specialty constraints only (removed travel time constraints)
for i in range(num_ers):
    for j in range(num_types):
        if m[i, j] == 0:  # If ER can't handle this emergency type
            constraints.append(x[i, j, :] == 0)

# ----------------------------
# 6. OBJECTIVE: Balanced capacity, travel time, and diversity
# ----------------------------


# Capacity slack terms
slacks = [C[i] - cp.sum(cp.multiply(x[i, :, :], lambda_jk)) for i in range(num_ers)]

# Travel time penalty term (element-wise multiplication with broadcasting)
travel_penalty = cp.sum(
    cp.multiply(
        cp.multiply(x, t_ik[:, np.newaxis, :]),  # (num_ers, num_types, num_locs)
        lambda_jk[np.newaxis, :, :]              # (1, num_types, num_locs)
    )
)

# Entropy regularization using CVXPY's entr atom
entropy = cp.sum(cp.entr(x))  # Equivalent to -x*log(x) but DCP-compliant

# Combined objective
objective = cp.Maximize(
    cp.sum(slacks)
    - alpha * travel_penalty
    + beta * entropy  # Tune coefficient for diversity
)



# ----------------------------
# 7. SOLVE & PRINT RESULTS
# ----------------------------

prob = cp.Problem(objective, constraints)
prob.solve(solver=cp.SCS, eps=1e-6, max_iters=10000)

print("Optimal routing fractions (x):")
for i, er in enumerate(ERs):
    for j, typ in enumerate(incident_types):
        for k, loc in enumerate(locations):
            if x.value[i, j, k] > 1e-3:
                print(f"Send {x.value[i, j, k]:.2f} of {typ} at {loc} to {er} (Travel time: {t_ik[i, k]} mins)")

print(f"\nObjective value (slack - α·travel): {prob.value:.2f}")

# ----------------------------
# 8. POST-SOLVE ANALYSIS
# ----------------------------

print("\n--- Key Metrics ---")
total_slack = sum(C[i] - np.sum(x.value[i, :, :] * lambda_jk) for i in range(num_ers))
total_travel = np.sum(x.value * t_ik[:, np.newaxis, :] * lambda_jk[np.newaxis, :, :])
print(f"Total capacity slack: {total_slack:.2f}")
print(f"Total weighted travel time: {total_travel:.2f} mins")

print("\n--- ER Utilization ---")
for i, er in enumerate(ERs):
    mu_i = np.sum(x.value[i, :, :] * lambda_jk)
    util = mu_i / C[i]
    print(f"{er}: {util:.1%} of capacity used ({mu_i:.1f}/{C[i]})")


[np.int64(10001), np.int64(10002), np.int64(10003), np.int64(10004), np.int64(10005)]


/usr/local/lib/python3.11/dist-packages/cvxpy/reductions/solvers/solving_chain_utils.py:30: UserWarning: The problem includes expressions that don't support CPP backend. Defaulting to the SCIPY backend for canonicalization.
  warnings.warn(UserWarning(


Optimal routing fractions (x):
Send 0.00 of CARDIAC CONDITION_1 at 10001 to 16211201 (Travel time: 13.599154876339757 mins)
Send 0.00 of CARDIAC CONDITION_1 at 10002 to 16211201 (Travel time: 13.20844146420109 mins)
Send 0.00 of CARDIAC CONDITION_1 at 10003 to 16211201 (Travel time: 13.154988548998354 mins)
Send 0.00 of CARDIAC CONDITION_1 at 10004 to 16211201 (Travel time: 13.300711383250114 mins)
Send 0.00 of CARDIAC CONDITION_1 at 10005 to 16211201 (Travel time: 13.087136694594008 mins)
Send 0.00 of CARDIAC CONDITION_2 at 10001 to 16211201 (Travel time: 13.599154876339757 mins)
Send 0.00 of CARDIAC CONDITION_2 at 10002 to 16211201 (Travel time: 13.20844146420109 mins)
Send 0.00 of CARDIAC CONDITION_2 at 10003 to 16211201 (Travel time: 13.154988548998354 mins)
Send 0.00 of CARDIAC CONDITION_2 at 10004 to 16211201 (Travel time: 13.300711383250114 mins)
Send 0.00 of CARDIAC CONDITION_2 at 10005 to 16211201 (Travel time: 13.087136694594008 mins)
Send 0.00 of CARDIAC CONDITION_3 at 10001

In [ ]:
# 3. Export results to CSV
x_optimal = x.value
print("Routing fractions array:\n", x_optimal)

# Convert to a list for JSON/CSV export
x_list = x_optimal.tolist()
data = []
for i in range(len(ERs)):
    for j in range(len(incident_types)):
        for k in range(len(locations)):
            if x_optimal[i, j, k] > 1e-3:
                data.append({
                    'ER': ERs[i],
                    'Incident Type': incident_types[j],
                    'Location': locations[k],
                    'Fraction': x_optimal[i, j, k]
                })
df = pd.DataFrame(data)
df.to_csv('/content/drive/MyDrive/Semester 2/Optimization /Project/output/FINALoptimal_routing2.csv', index=False)
print("Results exported to optimal_routing.csv")

# 4. Print objective value
print(f"\nObjective value: {prob.value:.2f}")

## Version 2 - Full Model

In [ ]:
!pip install scs[cuda]


In [ ]:
import pandas as pd
import numpy as np
import cvxpy as cp
from scipy.stats import norm

# ----------------------------
# 1. LOAD DATA
# ----------------------------
output_dir = "/content/drive/MyDrive/Semester 2/Optimization /Project/output"
hospitals = pd.read_csv(f"{output_dir}/new_hospitals_cleaned.csv")
lambda_stats = pd.read_csv(f"{output_dir}/new_lambda_stats.csv")
travel_times = pd.read_csv(f"{output_dir}/new_travel_times.csv")

# ----------------------------
# 2. BUILD INDEX SETS
# ----------------------------
ERs = list(hospitals["ID"])
incident_types = sorted(lambda_stats["EMERGENCY_TYPE"].unique())
locations = sorted(lambda_stats["ZIPCODE"].unique())

# Convert to 5-digit strings and deduplicate
locations = list(map(lambda x: str(x).zfill(5), locations))
locations = sorted(list(set(locations)))

num_ers = len(ERs)
num_types = len(incident_types)
num_locs = len(locations)

# ----------------------------
# 3. VALIDATION CHECKS
# ----------------------------
# Ensure ZIP codes exist in all datasets
lambda_stats["ZIPCODE"] = lambda_stats["ZIPCODE"].astype(str).str.zfill(5)
travel_times["ZIPCODE"] = travel_times["ZIP"].astype(str).str.zfill(5)

missing_in_lambda = set(lambda_stats["ZIPCODE"]) - set(locations)
missing_in_travel = set(travel_times["ZIPCODE"]) - set(locations)

if missing_in_lambda or missing_in_travel:
    raise ValueError(f"""
    Missing ZIP codes in locations list:
    - From lambda_stats: {missing_in_lambda}
    - From travel_times: {missing_in_travel}
    """)

# ----------------------------
# 4. PARAMETERS
# ----------------------------
# Capacities
C = hospitals.set_index("ID")["ER_CAPACITY"].reindex(ERs).values

# Poisson rates matrix
lambda_jk = np.zeros((num_types, num_locs))
for idx, row in lambda_stats.iterrows():
    j = incident_types.index(row["EMERGENCY_TYPE"])
    k = locations.index(str(row["ZIPCODE"]).zfill(5))
    lambda_jk[j, k] = row["LAMBDA"]

# Travel time matrix
t_ik = np.full((num_ers, num_locs), np.inf)
for idx, row in travel_times.iterrows():
    i = ERs.index(row["HOSPITAL_ID"])
    k = locations.index(str(row["ZIPCODE"]).zfill(5))
    t_ik[i, k] = row["TRAVEL_TIME_MINUTES"]

# Specialty compatibility
m = np.ones((num_ers, num_types), dtype=int)

# Hyperparameters
alpha = 0.001
beta = 0.1

# ----------------------------
# 5. DECISION VARIABLES & CONSTRAINTS
# ----------------------------
x = cp.Variable((num_ers, num_types, num_locs), nonneg=True)

constraints = []
for j in range(num_types):
    for k in range(num_locs):
        constraints.append(cp.sum(x[:, j, k]) == 1)

for i in range(num_ers):
    for j in range(num_types):
        if m[i, j] == 0:
            constraints.append(x[i, j, :] == 0)

# ----------------------------
# 6. OBJECTIVE FUNCTION
# ----------------------------
slacks = [C[i] - cp.sum(cp.multiply(x[i, :, :], lambda_jk)) for i in range(num_ers)]
travel_penalty = cp.sum(cp.multiply(
    cp.multiply(x, t_ik[:, np.newaxis, :]),  # Shape: (num_ers, num_types, num_locs)
    lambda_jk[np.newaxis, :, :]              # Shape: (1, num_types, num_locs)
))
entropy = cp.sum(cp.entr(x))

objective = cp.Maximize(cp.sum(slacks) - alpha * travel_penalty + beta * entropy)

# ----------------------------
# 7. SOLVE & RESULTS
# ----------------------------
prob = cp.Problem(objective, constraints)
# try:
#     prob.solve(solver=cp.SCS, eps=1e-6, max_iters=10000, gpu=True)
# except TypeError:
#     print("Falling back to CPU solve.")
prob.solve(solver=cp.SCS, eps=1e-6, max_iters = 10000)


print("Optimal routing fractions (x):")
for i, er in enumerate(ERs):
    for j, typ in enumerate(incident_types):
        for k, loc in enumerate(locations):
            if x.value[i, j, k] > 1e-3:
                print(f"Send {x.value[i, j, k]:.2f} of {typ} at {loc} to {er}")

print(f"\nObjective value: {prob.value:.2f}")


/usr/local/lib/python3.11/dist-packages/cvxpy/reductions/solvers/solving_chain_utils.py:30: UserWarning: The problem includes expressions that don't support CPP backend. Defaulting to the SCIPY backend for canonicalization.
  warnings.warn(UserWarning(


KeyboardInterrupt: 

### Version 2-> attempted to reduce computation time

In [ ]:
import pandas as pd
import numpy as np
import cvxpy as cp
from scipy.stats import norm

# ----------------------------
# 1. LOAD DATA
# ----------------------------
output_dir = "/content/drive/MyDrive/Semester 2/Optimization /Project/output"
hospitals = pd.read_csv(f"{output_dir}/new_hospitals_cleaned.csv")
lambda_stats = pd.read_csv(f"{output_dir}/new_lambda_stats.csv")
travel_times = pd.read_csv(f"{output_dir}/new_travel_times.csv")

# ----------------------------
# 2. BUILD INDEX SETS
# ----------------------------
ERs = list(hospitals["ID"])
incident_types = sorted(lambda_stats["EMERGENCY_TYPE"].unique())
locations = sorted(lambda_stats["ZIPCODE"].unique())

# Convert to 5-digit strings and deduplicate
locations = list(map(lambda x: str(x).zfill(5), locations))
locations = sorted(list(set(locations)))

num_ers = len(ERs)
num_types = len(incident_types)
num_locs = len(locations)

# ----------------------------
# 3. VALIDATION CHECKS
# ----------------------------
# Ensure ZIP codes exist in all datasets
lambda_stats["ZIPCODE"] = lambda_stats["ZIPCODE"].astype(str).str.zfill(5)
travel_times["ZIPCODE"] = travel_times["ZIP"].astype(str).str.zfill(5)

missing_in_lambda = set(lambda_stats["ZIPCODE"]) - set(locations)
missing_in_travel = set(travel_times["ZIPCODE"]) - set(locations)

if missing_in_lambda or missing_in_travel:
    raise ValueError(f"""
    Missing ZIP codes in locations list:
    - From lambda_stats: {missing_in_lambda}
    - From travel_times: {missing_in_travel}
    """)

# ----------------------------
# 4. PARAMETERS
# ----------------------------
# Capacities
C = hospitals.set_index("ID")["ER_CAPACITY"].reindex(ERs).values

# Poisson rates matrix
lambda_jk = np.zeros((num_types, num_locs))
for idx, row in lambda_stats.iterrows():
    j = incident_types.index(row["EMERGENCY_TYPE"])
    k = locations.index(str(row["ZIPCODE"]).zfill(5))
    lambda_jk[j, k] = row["LAMBDA"]

# Travel time matrix
t_ik = np.full((num_ers, num_locs), np.inf)
for idx, row in travel_times.iterrows():
    i = ERs.index(row["HOSPITAL_ID"])
    k = locations.index(str(row["ZIPCODE"]).zfill(5))
    t_ik[i, k] = row["TRAVEL_TIME_MINUTES"]

# Specialty compatibility
m = np.ones((num_ers, num_types), dtype=int)

# Hyperparameters
alpha = 0.001
beta = 0.1

# ----------------------------
# 5. DECISION VARIABLES & CONSTRAINTS
# ----------------------------
x = cp.Variable((num_ers, num_types, num_locs), nonneg=True)

constraints = []
for j in range(num_types):
    for k in range(num_locs):
        constraints.append(cp.sum(x[:, j, k]) == 1)

for i in range(num_ers):
    for j in range(num_types):
        if m[i, j] == 0:
            constraints.append(x[i, j, :] == 0)

# ----------------------------
# 6. PRECOMPUTE OBJECTIVE COMPONENTS
# ----------------------------
# Precompute travel time × lambda once
travel_penalty_weights = t_ik[:, np.newaxis, :] * lambda_jk[np.newaxis, :, :]
entropy_mask = (lambda_jk > 0).astype(float)

# Reformulated objective
travel_penalty = cp.sum(cp.multiply(x, travel_penalty_weights))
entropy = cp.sum(cp.multiply(cp.entr(x), entropy_mask[np.newaxis, :, :]))
slacks = [C[i] - cp.sum(cp.multiply(x[i, :, :], lambda_jk)) for i in range(num_ers)]

# ----------------------------
# 7. OBJECTIVE FUNCTION
# ----------------------------
objective = cp.Maximize(cp.sum(slacks) - alpha * travel_penalty + beta * entropy)

# ----------------------------
# 8. SOLVE & RESULTS
# ----------------------------
prob = cp.Problem(objective, constraints)
# try:
#     prob.solve(solver=cp.SCS, eps=1e-6, max_iters=10000, gpu=True)
# except TypeError:
#     print("Falling back to CPU solve.")
prob.solve(solver=cp.SCS, eps=1e-6, max_iters = 10000)

# ----------------------------
# 9. OUTPUT RESULTS
# ----------------------------
print("Optimal routing fractions (x):")
for i, er in enumerate(ERs):
    for j, typ in enumerate(incident_types):
        for k, loc in enumerate(locations):
            if x.value[i, j, k] > 1e-3:
                print(f"Send {x.value[i, j, k]:.2f} of {typ} at {loc} to {er}")

print(f"\nObjective value: {prob.value:.2f}")
# Optional: Print per-ER utilization for a sanity check.
used_capacity = np.array([np.sum(x.value[i, :, :] * lambda_jk) for i in range(num_ers)])
utilization = used_capacity / C
for i, er in enumerate(ERs):
    print(f"Utilization for {er}: {utilization[i]:.2f}")

/usr/local/lib/python3.11/dist-packages/cvxpy/reductions/solvers/solving_chain_utils.py:30: UserWarning: The problem includes expressions that don't support CPP backend. Defaulting to the SCIPY backend for canonicalization.
  warnings.warn(UserWarning(


# Visualizations

In [ ]:
import geopandas as gpd
import numpy as np

def simplify_geojson(input_url, output_path, tolerance=0.0001):
    """Simplify GeoJSON geometry using Douglas-Peucker algorithm"""
    print(f"Simplifying GeoJSON from {input_url}...")

    # Load original GeoJSON
    gdf = gpd.read_file(input_url)

    # Count vertices before simplification
    def count_vertices(geom):
        if geom.geom_type == 'Polygon':
            return len(geom.exterior.coords)
        elif geom.geom_type == 'MultiPolygon':
            return sum(len(poly.exterior.coords) for poly in geom.geoms)
        return 0

    original_vertices = gdf.geometry.apply(count_vertices).sum()
    print(f"Original vertices: {original_vertices}")

    # Simplify geometries
    gdf.geometry = gdf.geometry.simplify(tolerance, preserve_topology=True)

    # Count vertices after simplification
    simplified_vertices = gdf.geometry.apply(count_vertices).sum()
    print(f"Simplified vertices: {simplified_vertices} (reduction: {100*(original_vertices-simplified_vertices)/original_vertices:.1f}%)")

    # Save simplified version
    gdf.to_file(output_path, driver='GeoJSON')
    print(f"Saved simplified GeoJSON to {output_path}")
    return output_path
# Add this to your preprocessing pipeline
simplified_geojson_path = simplify_geojson(
    input_url="https://raw.githubusercontent.com/fedhere/PUI2015_EC/master/mam1612_EC/nyc-zip-code-tabulation-areas-polygons.geojson",
    output_path="/content/drive/MyDrive/Semester 2/Optimization /Project/output/nyc_zips_simplified.geojson",
    tolerance=0.0005  # Adjust based on needed detail (higher = more simplification)
)


In [ ]:
import pandas as pd
import plotly.graph_objects as go

# Load data
output_dir = "/content/drive/MyDrive/Semester 2/Optimization /Project/output"
routing_df = pd.read_csv(f"{output_dir}/FINALoptimal_routing2.csv")
hospitals = pd.read_csv(f"{output_dir}/clean_hospital_data.csv")

# Filter data for specific hospital
hospital_id = 20211206
hospital_data = routing_df[routing_df['ER'] == hospital_id]
hospital_info = hospitals[hospitals['ID'] == hospital_id]

# Get unique illness types
illness_types = sorted(hospital_data['Incident Type'].unique())

# Load GeoJSON
nyc_geojson_url = "https://raw.githubusercontent.com/fedhere/PUI2015_EC/master/mam1612_EC/nyc-zip-code-tabulation-areas-polygons.geojson"

# Create figure
fig = go.Figure()

# Add hospital marker (always visible, trace 0)
fig.add_trace(go.Scattermapbox(
    lat=hospital_info['LATITUDE'],
    lon=hospital_info['LONGITUDE'],
    mode='markers',
    marker=dict(size=20, color='red', symbol='circle'),
    name="Hospital",
    hoverinfo="text",
    text=f"Hospital {hospital_id} ({hospital_info['ER_CAPACITY'].values[0]} beds)"
))

# Create heatmap traces for each illness type
traces = []
for illness in illness_types:
    illness_df = hospital_data[hospital_data['Incident Type'] == illness]
    traces.append(go.Choroplethmapbox(
        geojson=nyc_geojson_url,
        locations=illness_df['Location'].astype(str),
        z=illness_df['Fraction'],
        colorscale="Viridis",
        zmin=0,
        zmax=1,
        marker_opacity=0.7,
        visible=False,
        name=illness,
        hovertemplate="<b>%{location}</b><br>Fraction: %{z:.2f}<extra></extra>",
        featureidkey="properties.postalCode"
    ))

# Add all heatmap traces (traces 1, 2, ...)
fig.add_traces(traces)

# Set first illness visible initially (trace 1)
if len(traces) > 0:
    fig.data[1].visible = True

# Create dropdown menu for illness types, always keeping hospital marker visible
dropdown_buttons = []
for i, illness in enumerate(illness_types):
    visibility = [True] + [False]*len(traces)
    visibility[i+1] = True  # Show the selected heatmap
    dropdown_buttons.append(dict(
        label=illness,
        method="update",
        args=[{"visible": visibility},
              {"title": f"Hospital {hospital_id} - {illness} Routing"}]
    ))

# Update layout for large window and dropdown in upper right
fig.update_layout(
    autosize=False,
    width=1200,
    height=800,
    margin=dict(l=30, r=30, t=30, b=30),
    mapbox_style="carto-positron",
    mapbox_zoom=11,
    mapbox_center=dict(
        lat=hospital_info['LATITUDE'].values[0],
        lon=hospital_info['LONGITUDE'].values[0]
    ),
    updatemenus=[dict(
        buttons=dropdown_buttons,
        direction="down",
        pad={"r": 10, "t": 10},
        showactive=True,
        x=1,          # Right side
        xanchor="right",
        y=1,          # Top
        yanchor="top"
    )],
    title=f"Hospital {hospital_id} - {illness_types[0]} Routing"
)

fig.show()


In [ ]:
!pip install "cvxpy[ECOS]"


In [ ]:
import cvxpy as cp
print(cp.installed_solvers())

In [ ]:
import cvxpy as cp

x = cp.Variable()
problem = cp.Problem(cp.Minimize(x**2), [x >= 1])
problem.solve(solver=cp.ECOS)  # Should work if ECOS is installed
print("Optimal value:", x.value)


\max \,\,
\underbrace{
    \sum_{i \in \mathcal{I}} \left( C_i - \sum_{j \in \mathcal{J}} \sum_{k \in \mathcal{K}} x_{ijk} \lambda_{jk} \right)
}_{\text{Capacity Utilization}}
- \alpha \underbrace{
    \sum_{i \in \mathcal{I}} \sum_{j \in \mathcal{J}} \sum_{k \in \mathcal{K}} x_{ijk} t_{ik} \lambda_{jk}
}_{\text{Response Efficiency}}
+ \beta \underbrace{
    \sum_{i \in \mathcal{I}} \sum_{j \in \mathcal{J}} \sum_{k \in \mathcal{K}} -x_{ijk} \log(x_{ijk})
}_{\text{Routing Diversity}}
